In [293]:
import pandas as pd
import numpy as np

import dash
from dash import dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

import pyodbc

import datetime

import warnings
warnings.filterwarnings("ignore")

# Data

In [294]:
dbname = "DW"
svname = "DESKTOP-IK0P52V"

In [295]:
conn_str = ("Driver={SQL Server Native Client 11.0};"
            f"Server={svname};"
            f"Database={dbname};"
            "Trusted_Connection=yes;")
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [296]:
def get_df_from_sql(filename):
    sql = f"SELECT * FROM [{dbname}].[dbo].[{filename}]"
    df = pd.read_sql(sql,conn)
    return df

In [297]:
f_movement = get_df_from_sql("Fact.Movement")
f_order = get_df_from_sql("Fact.Order")
f_purchase = get_df_from_sql("Fact.Purchase")
f_sale = get_df_from_sql("Fact.Sale")
f_stockholding = get_df_from_sql("Fact.StockHolding")
f_transaction = get_df_from_sql("Fact.Transaction")

d_city = get_df_from_sql("Dim_City")
d_customer = get_df_from_sql("Dim_Customer")
d_date = get_df_from_sql("Dim_Date")
d_employee = get_df_from_sql("Dim_Employee")
d_payment_method = get_df_from_sql("Dim_Payment_method")
d_stock_item = get_df_from_sql("Dim_Stock_item")
d_supplier = get_df_from_sql("Dim_Supplier")
d_transaction_type = get_df_from_sql("Dim_Transaction_type")

In [298]:
conn.close()

## Cleanse data

In [299]:
state_code = pd.read_csv("https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv")
state_code = state_code.rename(columns={' st':"st",' stusps':"stusps"})
state_code["stusps"] = state_code["stusps"].apply(lambda x: x[1:])
state_code.loc[51] = ["Puerto Rico (US Territory)",72,"PR"]
state_code.loc[52] = ["Virgin Islands (US Territory)",78,"VI"]
state_code["st"] = state_code["st"].apply(str)
lst = []
for i in range(len(state_code)):
    if len(state_code.loc[i,"st"]) == 1:
        lst.append("0"+state_code.loc[i,"st"])
    else:
        lst.append(state_code.loc[i,"st"])
state_code.st = lst

d_city.StateProvince = d_city.StateProvince.replace("Massachusetts[E]","Massachusetts")
d_city_new = d_city.merge(state_code,left_on=d_city.StateProvince,right_on=state_code.stname).drop(["key_0",'stname'],axis=1)

In [300]:
d_customer.BuyingGroup = d_customer.BuyingGroup.apply(lambda x: str(x).replace(" ",""))
d_customer.Category = d_customer.Category.apply(lambda x: str(x).replace(" ",""))

d_stock_item.SellingPackage = d_stock_item.SellingPackage.replace("None",None)
d_stock_item.BuyingPackage = d_stock_item.BuyingPackage.replace("None",None)
d_stock_item.Brand = d_stock_item.Brand.replace("None",None)
d_stock_item.Size = d_stock_item.Size.replace("None",None)
d_stock_item.Color = d_stock_item.Color.apply(lambda x: str(x).replace(" ",""))
d_stock_item.Color = d_stock_item.Color.replace("None",None)

## Func

In [301]:
def cutoff_long_label(df,field,n):
    for i in range(len(df[field])):
        if len(df.loc[i,field]) > n:
            df.loc[i,f"{field}_short"] = df.loc[i,field][:n]+"..."
        else:
            df.loc[i,f"{field}_short"] = df.loc[i,field]

In [302]:
def convert_none(series): 
    a = series.unique()
    for i in range(len(a)):
        a[i] = str(a[i])
    return a

In [303]:
def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '%.2f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

In [304]:
def waterfall_table(df,x_axis,hue,y_axis):
    df = df.groupby([x_axis,hue]).sum().reset_index()
    df = df.merge(df.groupby(x_axis).sum().reset_index().rename(columns={y_axis:"Subtotal"}),on=x_axis).sort_values(["Subtotal",hue], ascending=[False, True])
    dff = pd.DataFrame(columns=df.columns)
    result = pd.DataFrame(columns=[hue,"Changing"])
    for i in range(len(df[x_axis].unique())):
        tmp = df[df[x_axis] == df[x_axis].unique()[i]].rename(columns={y_axis:f"{y_axis}{i}"})
        if i == 0:
            dff = tmp
        else:
            dff = dff.merge(tmp,on=hue)
            dff["Changing"] = dff[f"{y_axis}{i}"] - dff[f"{y_axis}{i-1}"]
            dff_ = dff[[hue,"Changing"]]
            dff_.loc[-1e6] = [df[x_axis].unique()[i-1],dff[f"{y_axis}{i-1}"].sum()]
            dff_.loc[1e6] = [df[x_axis].unique()[i],dff[f"{y_axis}{i}"].sum()]
            dff_ = dff_.sort_index().reset_index(drop=True)
            result = pd.concat([result,dff_]).drop_duplicates().reset_index(drop=True)
    lst = []
    for i in range(len(result)):
        if i == 0:
            lst.append("absolute")
        elif (result.loc[i,hue] in df[x_axis].unique()):
            lst.append("total")
        else:
            lst.append("relative")
    result["Measure"] = lst

    lst = []
    for i in range(len(result)):
        if (result.loc[i,hue] in df[x_axis].unique()):
            lst.append(result.loc[i,hue])
        else:
            lst.append(None)
    result["x1"] = lst
    result['x1'] = result['x1'].bfill()
    result["ChangingText"] = result.Changing.apply(human_format)

    lst = []
    for i in range(len(result)):
        if (result.loc[i,hue] in df[x_axis].unique()):
            lst.append(result.loc[i,"Changing"])
        else:
            lst.append(lst[i-1]+result.loc[i,"Changing"])
    result["Cumsum"] = lst

    for i in range(len(result)):
        if i == 0:
            result.loc[i,"x2"] = 'Initial'
        elif (result.loc[i,hue] in df[x_axis].unique()):
            result.loc[i,"x2"] = 'Total'
        else:
            result.loc[i,"x2"] = result.loc[i,hue]
    result["Cumsum"] = lst
    return result

# Initialize

In [305]:
load_figure_template("lumen")
pio.templates.default = "lumen+seaborn"

In [306]:
app = JupyterDash(external_stylesheets=[dbc.themes.LUMEN])

## Header

In [307]:
PLOTLY_LOGO = "https://images.plot.ly/logo/new-branding/plotly-logomark.png"
header = dbc.Card(dbc.Row([
        html.H1(html.Span([html.Img(src=PLOTLY_LOGO, height="40px",className='me-2'),"DEMO"],className='d-flex align-items-center'),className = 'm-3'
    )
    ]))

In [308]:
# header = dbc.Card(html.H1("DEMO",className = 'm-3'))

## Slicer

### Slicer tab 1

In [309]:
### d_city
slicer_tab1 = html.Div([
    dbc.Row([
      dbc.Col([
        html.H6("Region"),
        dcc.Dropdown(d_city.Region.unique(),value=[],id="dropdown1_tab1",multi=True),
      ]),
      dbc.Col([
        html.H6("Subregion"),
        dcc.Dropdown(d_city.Subregion.unique(),value=[],id="dropdown2_tab1",multi=True)
      ]),    
        dbc.Col([
        html.H6("Country"),
        dcc.Dropdown(d_city.Country.unique(),value=[],id="dropdown3_tab1",multi=True)
      ]),
    ], className='m-2'),
    dbc.Row([
        dbc.Col([
        html.H6("Sales Territory"),
        dcc.Dropdown(d_city.SalesTerritory.unique(),value=[],id="dropdown4_tab1",multi=True)
      ]),    
        dbc.Col([
        html.H6("State Province"),
        dcc.Dropdown(d_city.StateProvince.unique(),value=[],id="dropdown5_tab1",multi=True)
      ]),  
    ], className='m-2'),
], className = 'mb-3')

### Slicer tab 2

In [310]:
### d_customer
slicer_tab2 = html.Div([
    dbc.Row(
      html.Div([
        html.H6(["Category",dcc.Checklist(["Multi-Select"],["Multi-Select"],id="all_tab2",style={"float":"right"},className="text-muted",labelClassName="me-2 form-check-label",inputClassName='form-check-input me-1 mt-0')]),
        dcc.Dropdown(d_customer.Category.unique(),value=list(d_customer.Category.unique()),id="dropdown_tab2",clearable=False,multi=True),
      ]), className='m-2'
    ),
    dbc.Row([
        html.H6("Buying Group"),
        dcc.Checklist(d_customer.BuyingGroup.unique(),list(d_customer.BuyingGroup.unique()),id="checklist_tab2",inline=True,labelClassName="me-2 form-check-label",inputClassName='form-check-input me-1')
    ], className='m-2')
], className = 'mb-3')

### Slicer tab 3

In [311]:
### d_date
slicer_tab3 = html.Div([
    dbc.Row([
        dbc.Col([
            html.H6("Date Range"), 
            dcc.DatePickerRange(
            start_date = datetime.date(2016,6,1),
            end_date = datetime.date.today(),
            min_date_allowed = d_date.Date.min(),
            max_date_allowed = d_date.Date.max(),
            clearable = True,
            number_of_months_shown = 3,
            display_format = "DD/MM/YYYY",
            id = "daterange"
            )
            ])], className = 'm-2'),
], className = 'mb-3')

### Slicer tab 4

In [312]:
### d_employee
slicer_tab4 = html.Div([
    dbc.Row([
        html.Span(["Employee Type",dcc.Checklist(["Salesperson","Picker"],["Salesperson","Picker"],id="checklist_tab4",inline=True,labelClassName="me-2 form-check-label",inputClassName='form-check-input me-1')],className='mb-1'),
        html.H6(["Employee Name",dcc.Checklist(["Multi-Select"],["Multi-Select"],id="all_tab4",style={"float":"right"},className="text-muted",labelClassName="me-2 form-check-label",inputClassName='form-check-input me-1 mt-0')]),
        dcc.Dropdown(d_employee.Employee.unique(),value=list(d_employee.Employee.unique()),id="dropdown_tab4",clearable=False,multi=True),
      ], className='m-2')
], className = 'mb-3')

### Slicer tab 5

In [313]:
### d_payment_method
slicer_tab5 = html.Div([
    dbc.Row([
        html.H6(["Payment Method",dcc.Checklist(["Multi-Select"],["Multi-Select"],id="all_tab5",style={"float":"right"},className="text-muted",labelClassName="me-2 form-check-label",inputClassName='form-check-input me-1 mt-0')]),
        dcc.Dropdown(d_payment_method.PaymentMethod.unique(),value=list(d_payment_method.PaymentMethod.unique()),id="dropdown_tab5",clearable=False,multi=True),
      ], className='m-2')
], className = 'mb-3')

### Slicer tab 6

In [314]:
### d_stock_item
slicer_tab6 = html.Div([
    dbc.Row([
        dbc.Col([
        html.H6("Color"),
        dcc.Dropdown(convert_none(d_stock_item.Color),value=None,id="dropdown1_tab6")
      ]),    
        dbc.Col([
        html.H6("Selling Package"),
        dcc.Dropdown(convert_none(d_stock_item.SellingPackage),value=None,id="dropdown2_tab6")
      ]),    
        dbc.Col([
        html.H6("Buying Package"),
        dcc.Dropdown(convert_none(d_stock_item.BuyingPackage),value=None,id="dropdown3_tab6")
      ]),    
        dbc.Col([
        html.H6("Brand"),
        dcc.Dropdown(convert_none(d_stock_item.Brand),value=None,id="dropdown4_tab6")
      ]),  
        dbc.Col([
        html.H6("Size"),
        dcc.Dropdown(convert_none(d_stock_item.Size),value=None,id="dropdown5_tab6")
      ]),
  ], className='m-2'),
    dbc.Row(
      html.Div([
        html.H6(["Stock Item",dcc.Checklist(["Multi-Select"],["Multi-Select"],id="all_tab6",style={"float":"right"},className="text-muted",labelClassName="me-2 form-check-label",inputClassName='form-check-input me-1 mt-0')]),
        dcc.Dropdown(d_stock_item.StockItem.unique(),value=[],id="dropdownmain_tab6",clearable=False,multi=True),
      ]), className='m-2'
  ),
], className = 'mb-3')

### Slicer tab 7

In [315]:
### d_supplier
slicer_tab7 = html.Div([
    dbc.Row([
        html.H6(["Supplier",dcc.Checklist(["Multi-Select"],["Multi-Select"],id="all_tab7",style={"float":"right"},className="text-muted",labelClassName="me-2 form-check-label",inputClassName='form-check-input me-1 mt-0')]),
        dcc.Dropdown(d_supplier.Supplier.unique(),value=list(d_supplier.Supplier.unique()),id="dropdown_tab7",clearable=False,multi=True),
      ], className='m-2')
], className = 'mb-3')

### Slicer tab 8

In [316]:
### d_transaction_type
slicer_tab8 = html.Div([
    dbc.Row([
        html.H6(["Transaction Type",dcc.Checklist(["Multi-Select"],["Multi-Select"],id="all_tab8",style={"float":"right"},className="text-muted",labelClassName="me-2 form-check-label",inputClassName='form-check-input me-1 mt-0')]),
        dcc.Dropdown(d_transaction_type.TransactionType.unique(),value=list(d_transaction_type.TransactionType.unique()),id="dropdown_tab8",clearable=False,multi=True),
      ], className='m-2')
], className = 'mb-3')

### Slicer Main

In [317]:
slicer = dbc.Card(dbc.Tabs([
    dbc.Tab(slicer_tab1,label="City",id='dtab1',active_label_style={"background-color":"#f0f0f0","color":"#000","font-weight":"bold","text-transform":"uppercase","font-size":"1rem"}),
    dbc.Tab(slicer_tab2,label="Customer",id='dtab2',active_label_style={"background-color":"#f0f0f0","color":"#000","font-weight":"bold","text-transform":"uppercase","font-size":"1rem"}),
    dbc.Tab(slicer_tab3,label="Date",id='dtab3',active_label_style={"background-color":"#f0f0f0","color":"#000","font-weight":"bold","text-transform":"uppercase","font-size":"1rem"}),
    dbc.Tab(slicer_tab4,label="Employee",id='dtab4',active_label_style={"background-color":"#f0f0f0","color":"#000","font-weight":"bold","text-transform":"uppercase","font-size":"1rem"}),
    dbc.Tab(slicer_tab5,label="Payment Method",id='dtab5',active_label_style={"background-color":"#f0f0f0","color":"#000","font-weight":"bold","text-transform":"uppercase","font-size":"1rem"}),
    dbc.Tab(slicer_tab6,label="Stock Item",id='dtab6',active_label_style={"background-color":"#f0f0f0","color":"#000","font-weight":"bold","text-transform":"uppercase","font-size":"1rem"}),
    dbc.Tab(slicer_tab7,label="Supplier",id='dtab7',active_label_style={"background-color":"#f0f0f0","color":"#000","font-weight":"bold","text-transform":"uppercase","font-size":"1rem"}),
    dbc.Tab(slicer_tab8,label="Transaction Type",id='dtab8',active_label_style={"background-color":"#f0f0f0","color":"#000","font-weight":"bold","text-transform":"uppercase","font-size":"1rem"}),
],id='dtabs',className="nav nav-tabs nav-fill"),
className="w-100",style={"height":"200px"})

### Collapse Slicer

In [318]:
collapse_slicer = html.Div(
    [
        dbc.Button("Slicer",id="collapse-button",className="btn btn-secondary my-2 me-2",n_clicks=0,),
        dbc.Button("Apply Filters",className = "btn btn-secondary my-2 ms-2", n_clicks=0 ,id = 'apply-button',style={"float": "right"}),
        dbc.Collapse(slicer,id="collapse")
    ]
)


## Card

### Card main

In [319]:
cardmain = ""

### Card tab 1

In [320]:
cardtext1 = dbc.Card([
    dbc.Row(
        html.Div([
            html.H4("Total Sales:"),
            html.H4(id="cardtext1_tab1"),
            html.H6(id="cardinfo1_tab1")
            ]),
            style={"height":"200px"},
            className="d-flex align-items-center text-center m-1",
        ),
    html.Hr(className='m-0'),
    
    dbc.Row(
        html.Div([
            html.H4("Total Profit:"),
            html.H4(id="cardtext2_tab1"),
            html.H6(id="cardinfo2_tab1")
            ]),
            style={"height":"200px"},
            className="d-flex align-items-center text-center m-1",
        ),
    html.Hr(className='m-0'),
    
    dbc.Row(
        html.Div([
            html.H4("Total Order Value:"),
            html.H4(id="cardtext3_tab1"),
            html.H6(id="cardinfo3_tab1")
            ]),
            style={"height":"200px"},
            className="d-flex align-items-center text-center m-1",
        ),
    html.Hr(className='m-0'),
    
    dbc.Row(
        html.Div([
            html.H4("Total Quantity Purchased:"),
            html.H4(id="cardtext4_tab1"),
            html.H6(id="cardinfo4_tab1")
            ]),
            style={"height":"200px"},
            className="d-flex align-items-center text-center m-1",
        ),
    html.Hr(className='m-0'),
    
    dbc.Row(
        html.Div([
            html.H4("Number of Transactions:"),
            html.H4(id="cardtext5_tab1"),
            html.H6(id="cardinfo5_tab1")
            ]),
            style={"height":"200px"},
            className="d-flex align-items-center text-center m-1",
        ),
])

### Card tab 2

In [321]:
cardtext2 = ""

### Card tab 3

In [322]:
cardtext3 = ""

### Card tab 4

In [323]:
cardtext4 = ""

### Card tab 5

In [324]:
cardtext5 = ""

### Card tab 6

In [325]:
cardtext6 = ""

## Graph

### Graph tab 1

In [326]:
### f_movement
graph_tab1 = html.Div([
    dbc.Row([
        dbc.Col(cardtext1,className="col-md-2 m-0 p-0", align='center'),
        dbc.Col([
                dbc.Row([
                    dbc.Col(dcc.Graph(style={"height":"500px"},id='graph1_tab1'),width=6),
                    dbc.Col(dcc.Graph(style={"height":"500px"},id='graph2_tab1'),width=6)
                    ]),
                html.Hr(className='my-2'),
                dbc.Row(dcc.Graph(style={"height":"500px"},id='graph3_tab1'))
                ],className="col-md-10 m-0"),
            ], className = "g-3 m-0")
    ])

### Graph tab 2

In [327]:
graph_tab2 = html.Div([
    dbc.Row([
        dbc.Col(cardtext2,className="col-md-2 m-0 p-0", align='center'),
        dbc.Col([
                dbc.Row([
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph1_tab2"),width=6),
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph2_tab2"),width=6),
                    ]),
                html.Hr(className='m-0'),
                dbc.Row([
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph3_tab2"),width=3),
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph4_tab2"),width=3),
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph5_tab2"),width=3),
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph6_tab2"),width=3)
                    ]),
                ],className="col-md-10 m-0"),
            ], className = "g-3 m-0")
])

### Graph tab 3

In [328]:
### f_purchase
graph_tab3 = html.Div([
dbc.Row([
        dbc.Col(cardtext3,className="col-md-2 m-0 p-0", align='center'),
        dbc.Col(dcc.Graph(style={"height":"1000px"},id="graph1_tab3"),className="col-md-5 m-0 p-0"),
        dbc.Col(dcc.Graph(style={"height":"1000px"},id="graph2_tab3"),className="col-md-5 m-0 p-0")
        ], align='center')
])

### Graph tab 4

In [329]:
### f_sale
graph_tab4 = html.Div([
    dbc.Row([
        dbc.Col(cardtext4,className="col-md-2 m-0 p-0", align='center'),
        dbc.Col([
                dbc.Row([
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph1_tab4"),className="col-md-6"),
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph2_tab4"),className="col-md-5"),
                    dbc.Col([
                        dcc.Dropdown(["Sales","Profit"],"Sales",clearable=False,id="switch_graph_gtab4"),
                    ],className="col-md-1"), 
                    ]),
                html.Hr(className='m-0'),
                dbc.Row([
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph3_tab4"),className="col-md-3"),
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph4_tab4"),className="col-md-3"),
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph5_tab4"),className="col-md-6"),
                    ]),
                ],className="col-md-10 m-0"),
            ], className = "g-3 m-0")
])

### Graph tab 5

In [330]:
### f_stock_holding
graph_tab5 = html.Div([
    dbc.Row([
        dbc.Col(cardtext5,className="col-md-2 m-0 p-0", align='center'),
        dbc.Col([
                dbc.Row(dcc.Graph(style={"height":"500px"},id="graph1_tab5")),
                html.Hr(className='m-0'),
                dbc.Row([
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph2_tab5"),width=6),
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph3_tab5"),width=6)
                    ]),
                ],className="col-md-10 m-0"),
            ], className = "g-3 m-0")
])

### Graph tab 6

In [331]:
### f_transaction
graph_tab6 = html.Div([
    dbc.Row([
        dbc.Col(cardtext6,className="col-md-2 m-0 p-0", align='center'),
        dbc.Col([
                dbc.Row([
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph1_tab6"),width=4),
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph2_tab6"),width=4),
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph3_tab6"),width=4)
                    ]),
                html.Hr(className='m-0'),
                dbc.Row([
                    dbc.Col(dcc.Graph(style={"height":"500px"},id="graph4_tab6")),
                    ]),
                ],className="col-md-10 m-0"),
            ], className = "g-3 m-0")
])

In [332]:
graph = dbc.Tabs([
    dbc.Tab(graph_tab1, label="Movement",id='ftab1',active_label_style={"background-color":"#158cba","color":"#fff","font-weight":"bold","text-transform":"uppercase","font-size":"1.05rem"},),
    dbc.Tab(graph_tab2, label="Order",id='ftab2',active_label_style={"background-color":"#158cba","color":"#fff","font-weight":"bold","text-transform":"uppercase","font-size":"1.05rem"},),
    dbc.Tab(graph_tab3, label="Purchase",id='ftab3',active_label_style={"background-color":"#158cba","color":"#fff","font-weight":"bold","text-transform":"uppercase","font-size":"1.05rem"},),
    dbc.Tab(graph_tab4, label="Sales",id='ftab4',active_label_style={"background-color":"#158cba","color":"#fff","font-weight":"bold","text-transform":"uppercase","font-size":"1.05rem"},),
    dbc.Tab(graph_tab5, label="Stock Holding",id='ftab5',active_label_style={"background-color":"#158cba","color":"#fff","font-weight":"bold","text-transform":"uppercase","font-size":"1.05rem"},),
    dbc.Tab(graph_tab6, label="Transaction",id='ftab6',active_label_style={"background-color":"#158cba","color":"#fff","font-weight":"bold","text-transform":"uppercase","font-size":"1.05rem"},),
],id='ftabs',className="nav nav-tabs nav-fill")

## Outer Tabs

In [333]:
outer_tabs = dbc.Tabs([
    dbc.Tab(cardmain,label="Overall",active_label_style={"background-color":"#f0f0f0","color":"#000","font-weight":"bold","text-transform":"uppercase","font-size":"1.1rem"},),
    dbc.Tab(graph,label="Detailed",active_label_style={"background-color":"#f0f0f0","color":"#000","font-weight":"bold","text-transform":"uppercase","font-size":"1.1rem"},),
])

# Layout

In [334]:
app.layout = dbc.Container([header,collapse_slicer,outer_tabs],className='dbc',fluid=True)

# Callback

## Sync tabs

In [335]:
@app.callback(
    Output('dtabs','active_tab'),
    Output('dtab1','disabled'),
    Output('dtab2','disabled'),
    Output('dtab3','disabled'),
    Output('dtab4','disabled'),
    Output('dtab5','disabled'),
    Output('dtab6','disabled'),
    Output('dtab7','disabled'),
    Output('dtab8','disabled'),
    Input('ftabs','active_tab'),
)
def config_tab(tabs):
    if tabs =="tab-0":
        return 'tab-1', True, False, False, True, True, False, False, False
    elif tabs =="tab-1":
        return 'tab-0', False, False, False, False, True, False, True, True
    elif tabs =="tab-2":
        return 'tab-5', True, True, False, True, True, False, False, True
    elif tabs =="tab-3":
        return 'tab-0', False, False, False, False, True, False, True, True
    elif tabs =="tab-4":
        return 'tab-5', True, True, True, True, True, False, True, True
    else:
        return 'tab-1', True, False, False, True, False, True, False, False

## Slicer

### Collapse Slicer

In [336]:
@app.callback(
    Output("collapse", "is_open"),
    [Input("collapse-button", "n_clicks")],
    [State("collapse", "is_open")]
)
def toggle_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

### Slicer Tab 1: Dependable Dropdown

In [337]:
@app.callback(
    Output('dropdown2_tab1', 'options'),
    Output('dropdown2_tab1', 'value'),
    Input('dropdown1_tab1','value'),
)
def update_dropdown(v1):
    if len(v1) == 0:
        temp = d_city
    else:
        temp = d_city[d_city['Region'].isin(v1)]
    return temp['Subregion'].unique(), []

@app.callback(
    Output('dropdown3_tab1', 'options'),
    Output('dropdown3_tab1', 'value'),
    Input('dropdown1_tab1','value'),
    Input('dropdown2_tab1','value'),
)
def update_dropdown(v1,v2):
    if (len(v1) == 0) & (len(v2) == 0):
        temp = d_city
    elif len(v2) == 0:
        temp = d_city[d_city['Region'].isin(v1)]
    else:
        temp = d_city[d_city['Region'].isin(v1)]
        temp = d_city[d_city['Subregion'].isin(v2)]
    return temp['Country'].unique(), []

@app.callback(
    Output('dropdown4_tab1', 'options'),
    Output('dropdown4_tab1', 'value'),
    Input('dropdown1_tab1','value'),
    Input('dropdown2_tab1','value'),
    Input('dropdown3_tab1','value'),
)
def update_dropdown(v1,v2,v3):
    if (len(v1) == 0) & (len(v2) == 0) & (len(v3) == 0):
        temp = d_city
    elif (len(v2) == 0) & (len(v3) == 0):
        temp = d_city[d_city['Region'].isin(v1)]
    elif len(v3) == 0:
        temp = d_city[d_city['Region'].isin(v1)]
        temp = d_city[d_city['Subregion'].isin(v2)]
    else:
        temp = d_city[d_city['Region'].isin(v1)]
        temp = d_city[d_city['Subregion'].isin(v2)]
        temp = d_city[d_city['Country'].isin(v3)]
    return temp['SalesTerritory'].unique(), []

@app.callback(
    Output('dropdown5_tab1', 'options'),
    Output('dropdown5_tab1', 'value'),
    Input('dropdown1_tab1','value'),
    Input('dropdown2_tab1','value'),
    Input('dropdown3_tab1','value'),
    Input('dropdown4_tab1','value'),
)
def update_dropdown(v1,v2,v3,v4):
    if (len(v1) == 0) & (len(v2) == 0) & (len(v3) == 0) & (len(v4) == 0):
        temp = d_city
    elif (len(v2) == 0) & (len(v3) == 0) & (len(v4) == 0):
        temp = d_city[d_city['Region'].isin(v1)]
    elif (len(v3) == 0) & (len(v4) == 0):
        temp = d_city[d_city['Region'].isin(v1)]
        temp = d_city[d_city['Subregion'].isin(v2)]
    elif len(v4) == 0:
        temp = d_city[d_city['Region'].isin(v1)]
        temp = d_city[d_city['Subregion'].isin(v2)]
        temp = d_city[d_city['Country'].isin(v3)]
    else:
        temp = d_city[d_city['Region'].isin(v1)]
        temp = d_city[d_city['Subregion'].isin(v2)]
        temp = d_city[d_city['Country'].isin(v3)]
        temp = d_city[d_city['SalesTerritory'].isin(v4)]
    return temp['StateProvince'].unique(), []

### Slicer Tab 2, 5, 7, 8: Allow Multi-Choice

In [338]:
@app.callback(Output('dropdown_tab2','multi'), Output('dropdown_tab2','options'), Output('dropdown_tab2','value'), Input('all_tab2','value'), Input('dropdown_tab2','value'),
              prevent_initial_call = True
              )
def multi_choice_button(on,value):
    input_id = ctx.triggered_id
    if on == ["Multi-Select"]:   
        return True, d_customer.Category.unique(), value
    else:
        if input_id == "all_tab2":
            if (type(value) == list) & (len(value) != 0):
                return False, d_customer.Category.unique(), value[0]
            elif (type(value) == list) & (len(value) == 0):
                return False, d_customer.Category.unique(), d_customer.Category.unique()[0]
            else:  
                return False, d_customer.Category.unique(), [value]
        else:
            return False, d_customer.Category.unique(), value
        
@app.callback(Output('dropdown_tab5','multi'), Output('dropdown_tab5','options'), Output('dropdown_tab5','value'), Input('all_tab5','value'), Input('dropdown_tab5','value'),
              prevent_initial_call = True
              )
def multi_choice_button(on,value):
    input_id = ctx.triggered_id
    if on == ["Multi-Select"]:   
        return True, d_payment_method.PaymentMethod.unique(), value
    else:
        if input_id == "all_tab5":
            if (type(value) == list) & (len(value) != 0):
                return False, d_payment_method.PaymentMethod.unique(), value[0]
            elif (type(value) == list) & (len(value) == 0):
                return False, d_payment_method.PaymentMethod.unique(), d_payment_method.PaymentMethod.unique()[0]
            else:  
                return False, d_payment_method.PaymentMethod.unique(), [value]
        else:
            return False, d_payment_method.PaymentMethod.unique(), value

@app.callback(Output('dropdown_tab7','multi'), Output('dropdown_tab7','options'), Output('dropdown_tab7','value'), Input('all_tab7','value'), Input('dropdown_tab7','value'),
              prevent_initial_call = True
              )
def multi_choice_button(on,value):
    input_id = ctx.triggered_id
    if on == ["Multi-Select"]:   
        return True, d_supplier.Supplier.unique(), value
    else:
        if input_id == "all_tab7":
            if (type(value) == list) & (len(value) != 0):
                return False, d_supplier.Supplier.unique(), value[0]
            elif (type(value) == list) & (len(value) == 0):
                return False, d_supplier.Supplier.unique(), d_supplier.Supplier.unique()[0]
            else:  
                return False, d_supplier.Supplier.unique(), [value]
        else:
            return False, d_supplier.Supplier.unique(), value

@app.callback(Output('dropdown_tab8','multi'), Output('dropdown_tab8','options'), Output('dropdown_tab8','value'), Input('all_tab8','value'), Input('dropdown_tab8','value'),
              prevent_initial_call = True
              )
def multi_choice_button(on,value):
    input_id = ctx.triggered_id
    if on == ["Multi-Select"]:   
        return True, d_transaction_type.TransactionType.unique(), value
    else:
        if input_id == "all_tab8":
            if (type(value) == list) & (len(value) != 0):
                return False, d_transaction_type.TransactionType.unique(), value[0]
            elif (type(value) == list) & (len(value) == 0):
                return False, d_transaction_type.TransactionType.unique(), d_transaction_type.TransactionType.unique()[0]
            else:  
                return False, d_transaction_type.TransactionType.unique(), [value]
        else:
            return False, d_transaction_type.TransactionType.unique(), value
        

### Slicer Tab 4: Allow Multi-Choice and Filter

In [339]:
@app.callback(Output('dropdown_tab4','multi'), 
              Output('dropdown_tab4','options'), 
              Output('dropdown_tab4','value'), 
              Input('all_tab4','value'), 
              Input('dropdown_tab4','value'), 
              Input('checklist_tab4','value'),
              prevent_initial_call = True
              )
def multi_choice_button_but_more_detailed(on,value,type_):
    input_id = ctx.triggered_id
    if type_ == ["Salesperson"]:
        dff = d_employee[d_employee.IsSalesperson == True]
        if on == ["Multi-Select"]:   
            return True, dff.Employee.unique(), value
        else:
            if input_id == "all_tab4":
                if (type(value) == list) & (len(value) != 0):
                    return False, dff.Employee.unique(), value[0]
                elif (type(value) == list) & (len(value) == 0):
                    return False, dff.Employee.unique(), dff.Employee.unique()[0]
                else:  
                    return False, dff.Employee.unique(), [value]
            else:
                return False, dff.Employee.unique(), value
    elif type_ == ["Picker"]:
        dff2 = d_employee[d_employee.IsSalesperson == False]
        if on == ["Multi-Select"]:   
            return True, dff2.Employee.unique(), value 
        else:
            if input_id == "all_tab4":
                if (type(value) == list) & (len(value) != 0):
                    return False, dff2.Employee.unique(), value[0]
                elif (type(value) == list) & (len(value) == 0):
                    return False, dff2.Employee.unique(), dff2.Employee.unique()[0]
                else:  
                    return False, dff2.Employee.unique(), [value]
            else:
                return False, dff2.Employee.unique(), value
    elif len(type_) == 2:
        if on == ["Multi-Select"]:   
            return True, d_employee.Employee.unique(), value
        else:
            if input_id == "all_tab4":
                if (type(value) == list) & (len(value) != 0):
                    return False, d_employee.Employee.unique(), value[0]
                elif (type(value) == list) & (len(value) == 0):
                    return False, d_employee.Employee.unique(), d_employee.Employee.unique()[0]
                else:  
                    return False, d_employee.Employee.unique(), [value]
            else:
                return False, d_employee.Employee.unique(), value
    else:
        return False, [], None

### Slicer Tab 6: Allow Multiple Choice + More Filters

In [340]:
@app.callback(
    Output("dropdownmain_tab6","multi"),
    Output("dropdownmain_tab6","options"),
    Output("dropdownmain_tab6","value"),
    Output("dropdownmain_tab6","disabled"),
    Input("dropdown1_tab6","value"),
    Input("dropdown2_tab6","value"),
    Input("dropdown3_tab6","value"),
    Input("dropdown4_tab6","value"),
    Input("dropdown5_tab6","value"),
    Input("all_tab6","value"),
    Input("dropdownmain_tab6","value"),
    prevent_initial_call = True
)
def stockitem_slicer(v1,v2,v3,v4,v5,on,v):
    dff = d_stock_item.copy()
    dff.Color = dff.Color.apply(str)
    dff.SellingPackage = dff.SellingPackage.apply(str)
    dff.BuyingPackage = dff.BuyingPackage.apply(str)
    dff.Brand = dff.Brand.apply(str)
    dff.Size = dff.Size.apply(str)
    if v1 is not None:
        dff = dff[dff.Color == v1]
    if v2 is not None:
        dff = dff[dff.SellingPackage == v2]
    if v3 is not None:
        dff = dff[dff.BuyingPackage == v3]
    if v4 is not None:
        dff = dff[dff.Brand == v4]
    if v5 is not None:
        dff = dff[dff.Size == v5]
    
    input_id = ctx.triggered_id
    if on == ["Multi-Select"]:   
        if len(dff) > 0:
            return True, dff.StockItem.unique(), v, False
        else:
            return True, d_stock_item.StockItem.unique(), None, True
    else:
        if len(dff) > 0:
            if input_id == "all_tab6":
                if v is None:
                    return True, d_stock_item.StockItem.unique(), None, True
                elif (type(v) == list) & (len(v) != 0):
                    return False, dff.StockItem.unique(), v[0], False
                elif (type(v) == list) & (len(v) == 0):
                    return False, dff.StockItem.unique(), dff.StockItem.unique()[0], False
                else:  
                    return False, dff.StockItem.unique(), v, False
            else:
                return False, dff.StockItem.unique(), v, False
        else:
            return True, d_stock_item.StockItem.unique(), None, True

## Cardtext

### Cardtext tab 1

In [341]:
@app.callback( 
    Output("cardtext1_tab1","children"),
    Output("cardinfo1_tab1","children"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'), 
    Input('apply-button','n_clicks')
)
def adjust_cardtext(sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)

    dff = f_sale.copy()
    dff = dff[(dff["InvoiceDateKey"] >= sdate) & (dff['InvoiceDateKey'] <= edate)]
    
    return f"${human_format(dff.TotalIncludingTax.sum())}", f"Avg ${human_format(dff.TotalIncludingTax.sum()/(abs((edate - sdate).days)))} per day"

@app.callback( 
    Output("cardtext2_tab1","children"),
    Output("cardinfo2_tab1","children"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'), 
    Input('apply-button','n_clicks')
)
def adjust_cardtext(sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)

    dff = f_sale.copy()
    dff = dff[(dff["InvoiceDateKey"] >= sdate) & (dff['InvoiceDateKey'] <= edate)]
    
    return f"${human_format(dff.Profit.sum())}", f"Avg ${human_format(dff.Profit.sum()/(abs((edate - sdate).days)))} per day"

@app.callback( 
    Output("cardtext3_tab1","children"),
    Output("cardinfo3_tab1","children"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'), 
    Input('apply-button','n_clicks')
)
def adjust_cardtext(sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)

    dff = f_order.copy()
    dff = dff[(dff["OrderDateKey"] >= sdate) & (dff['OrderDateKey'] <= edate)]
    
    return f"${human_format(dff.TotalIncludingTax.sum())}", f"Avg ${human_format(dff.TotalIncludingTax.sum()/(abs((edate - sdate).days)))} per day"

@app.callback( 
    Output("cardtext4_tab1","children"),
    Output("cardinfo4_tab1","children"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'), 
    Input('apply-button','n_clicks')
)
def adjust_cardtext(sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)

    dff = f_purchase.copy()
    dff = dff[(dff["DateKey"] >= sdate) & (dff['DateKey'] <= edate)]
    
    return f"{human_format(dff.OrderedQuantity.sum())}", f"Avg {human_format(dff.OrderedQuantity.sum()/(abs((edate - sdate).days)))} per day"

@app.callback( 
    Output("cardtext5_tab1","children"),
    Output("cardinfo5_tab1","children"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'), 
    Input('apply-button','n_clicks')
)
def adjust_cardtext(sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)

    dff = f_transaction.copy()
    dff = dff[(dff["DateKey"] >= sdate) & (dff['DateKey'] <= edate)]
    
    return f"{human_format(len(dff))}", f"Avg {human_format(len(dff)/(abs((edate - sdate).days)))} per day"

### Cardtext tab 2

### Cardtext tab 3

### Cardtext tab 4

### Cardtext tab 5

### Cardtext tab 6

## Plot

### Plot tab 1

#### Graph 1

In [342]:
@app.callback(
    Output("graph1_tab1","figure"),
    State("checklist_tab2","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks')
)
def update_graph(buyinggroup,sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_customer.copy()
    temp1 = temp1[temp1["BuyingGroup"].isin(buyinggroup)]
    a = f_movement.merge(temp1,left_on=f_movement.CustomerHash, right_on=temp1.hash)[["DateKey","BuyingGroup","Quantity"]]
    b = a.groupby(["BuyingGroup","DateKey"]).sum().reset_index()
    date_temp = d_date[(d_date["Date"] >= sdate) & (d_date["Date"] <= edate)]
    c = b.BuyingGroup.drop_duplicates().to_frame().merge(date_temp.Date,how='cross')
    df_plot = c.merge(b,how='left',left_on=["BuyingGroup","Date"],right_on=["BuyingGroup","DateKey"]).drop("DateKey",axis=1).fillna(0)
    df_plot['YM'] = pd.to_datetime(df_plot["Date"]).dt.strftime("%Y-%m")
    df_plot = df_plot.groupby(["BuyingGroup","YM"]).sum().reset_index()
    fig = px.area(df_plot,x="YM",y="Quantity",color="BuyingGroup",pattern_shape="BuyingGroup")
    fig.update_layout(
        legend=dict(
            yanchor="top",
            y=1.15,
            xanchor="right",
            x=0.99
            ),
        font={"size":14}, xaxis_title=None,
        title = "Stock Movement Quantity Over Time",title_y = 0.95,title_x = 0.5,
        legend_title_font_size = 12,
        hovermode='x unified'
    )
    return fig

#### Graph 2

In [343]:
@app.callback(
    Output("graph2_tab1","figure"),
    State("dropdown1_tab6","value"),
    State("dropdown2_tab6","value"),
    State("dropdown3_tab6","value"),
    State("dropdown4_tab6","value"),
    State("dropdown5_tab6","value"),
    State("dropdownmain_tab6","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks')
)
def update_graph(v1,v2,v3,v4,v5,v6,sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_stock_item.copy()
    temp1.Color = temp1.Color.apply(str)
    temp1.SellingPackage = temp1.SellingPackage.apply(str)
    temp1.BuyingPackage = temp1.BuyingPackage.apply(str)
    temp1.Brand = temp1.Brand.apply(str)
    temp1.Size = temp1.Size.apply(str)
    temp1.StockItem = temp1.StockItem.apply(str)
    if v1 is not None:
        temp1 = temp1[temp1.Color == v1]
    if v2 is not None:
        temp1 = temp1[temp1.SellingPackage == v2]
    if v3 is not None:
        temp1 = temp1[temp1.BuyingPackage == v3]
    if v4 is not None:
        temp1 = temp1[temp1.Brand == v4]
    if v5 is not None:
        temp1 = temp1[temp1.Size == v5]
    if (type(v6) == list) & (len(v6) == 0):
        pass
    elif type(v6) == list:
        temp1 = temp1[temp1.StockItem.isin(v6)]
    else:
        temp1 = temp1[temp1.StockItem == v6]
    date_temp = d_date[(d_date["Date"] >= sdate) & (d_date["Date"] <= edate)]
    temp2 = f_movement.merge(date_temp,left_on=f_movement.DateKey,right_on=date_temp.Date)[["Date","Quantity","StockItemHash"]]
    a = temp2.merge(temp1,left_on=temp2.StockItemHash,right_on=temp1.hash)[["Date","Quantity","SellingPackage","BuyingPackage"]]
    a["Sell-Buy Package"] = a.SellingPackage + " - " + a.BuyingPackage
    a['YQ'] = pd.to_datetime(a["Date"]).dt.year.apply(str) + "-Q" + pd.to_datetime(a["Date"]).dt.quarter.apply(str)
    
    top = a.groupby("Sell-Buy Package").sum()["Quantity"].reset_index().sort_values("Quantity",ascending=False)
    if len(top) > 3:
        top = top.head(3)
    a = a[a["Sell-Buy Package"].isin(top["Sell-Buy Package"])]
    df_plot_tmp = a.groupby(["Sell-Buy Package","YQ"]).sum()["Quantity"].reset_index()
    date_temp['YQ'] = pd.to_datetime(date_temp["Date"]).dt.year.apply(str) + "-Q" + pd.to_datetime(date_temp["Date"]).dt.quarter.apply(str)
    
    tmp = date_temp['YQ'].unique()
    lst = []
    for item in df_plot_tmp["Sell-Buy Package"].unique():
        for ym in tmp:
            lst.append([item,ym])
    dff = pd.DataFrame(lst,columns=['Sell-Buy Package',"YQ"])
    df_plot = dff.merge(df_plot_tmp,how='left',on=['Sell-Buy Package',"YQ"]).fillna(0)
    fig = px.line(df_plot,df_plot.YQ,df_plot.Quantity,color=df_plot["Sell-Buy Package"])
    fig.update_layout(
                legend=dict(
                    yanchor="top",
                    y=1.15,
                    xanchor="right",
                    x=0.99
                    ),
                font={"size":14}, xaxis_title=None,
                title = "Top 3 Sell-Buy Package Type <br>with Highest Stock Movement Quantity",title_y = 0.95,title_x = 0.5,
                legend_title_font_size = 12,
                hovermode='x unified')
    return fig

#### Graph 3

In [344]:
@app.callback(
    Output("graph3_tab1","figure"),
    State("dropdown_tab7","value"),
    State("dropdown_tab8","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks')
)
def update_graph(sup,trans,sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_transaction_type.copy()
    if type(trans) == list:
            temp1 = temp1[temp1.TransactionType.isin(trans)]
    else:
        temp1 = temp1[temp1.TransactionType == trans]
    temp2 = d_supplier.copy()
    if type(sup) == list:
        temp2 = temp2[temp2.Supplier.isin(sup)]
    else:
        temp2 = temp2[temp2.Supplier == sup]
    temp3 = f_movement.copy()
    temp3 = temp3[(temp3["DateKey"] >= sdate) & (temp3["DateKey"] <= edate)]
    a = temp3.merge(temp1,left_on=temp3.TransactionTypeHash,right_on=temp1.hash)[["TransactionType","SupplierHash","Quantity"]]
    b = a.merge(temp2,left_on=a.SupplierHash,right_on=temp2.hash)[["TransactionType","Supplier","Quantity"]]
    df_plot = b.groupby(["Supplier","TransactionType"]).sum().reset_index().sort_values(["TransactionType","Supplier"],ascending=[True,False]).reset_index(drop=True)
    
    fig = px.bar_polar(df_plot, 
                       r="Quantity", 
                       theta="Supplier",
                       color="TransactionType", 
                       )
    fig.update_layout(
                font={"size":14},
                title = "Stock Movement Quantity by Supplier",title_y = 1,title_x = 0.5,
    )
    return fig

### Plot tab 2

#### Graph 1

In [345]:
@app.callback(
    Output("graph1_tab2","figure"),
    State('dropdown1_tab1', 'value'), 
    State('dropdown2_tab1', 'value'), 
    State('dropdown3_tab1', 'value'), 
    State('dropdown4_tab1', 'value'), 
    State('dropdown5_tab1', 'value'), 
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    State("dropdown_tab2","value"),
    State("checklist_tab2","value"),
    Input('apply-button','n_clicks'),
)    
def update_graph(v1,v2,v3,v4,v5,sdate,edate,vtab2_1,vtab2_2,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_city_new.copy()
    if len(v1) != 0:
        temp1 = temp1[temp1["Region"].isin(v1)]
    if len(v2) != 0:    
        temp1 = temp1[temp1["Subregion"].isin(v2)]
    if len(v3) != 0:    
        temp1 = temp1[temp1["Country"].isin(v3)]
    if len(v4) != 0:    
        temp1 = temp1[temp1["SalesTerritory"].isin(v4)]
    if len(v5) != 0:    
        temp1 = temp1[temp1["StateProvince"].isin(v5)]
    
    date_tmp = d_date.copy()
    date_tmp = date_tmp[(date_tmp["Date"] >= sdate) & (date_tmp["Date"] <= edate)]
    date_tmp['YM'] = pd.to_datetime(date_tmp["Date"]).dt.strftime("%Y-%m")
    temp2 = f_order.copy()
    temp2 = temp2[(temp2["OrderDateKey"] >= sdate) & (temp2["OrderDateKey"] <= edate)]
    
    temp3 = d_customer.copy()
    if type(vtab2_1) == list: 
        temp3 = temp3[temp3.Category.isin(vtab2_1)]
    else:
        temp3 = temp3[temp3.Category == vtab2_1]
    if len(vtab2_2) == 0:
        raise dash.exceptions.PreventUpdate
    else: 
        temp3 = temp3[temp3.BuyingGroup.isin(vtab2_2)]
    a = temp2.merge(temp1,left_on=temp2.CityHash,right_on=temp1.hash)[["StateProvince","st","stusps","CustomerHash","TotalIncludingTax",'OrderDateKey']]
    b = a.merge(temp3,left_on=a.CustomerHash,right_on=temp3.hash)[["Customer","StateProvince","st","stusps","TotalIncludingTax",'OrderDateKey']]
    c = b.groupby(["StateProvince","st","stusps",'OrderDateKey']).sum().reset_index()
    c['YM'] = pd.to_datetime(c["OrderDateKey"]).dt.strftime("%Y-%m")
    
    df_plot = c.groupby(["StateProvince","st","stusps",'YM']).sum().reset_index()
    g1 = df_plot[["StateProvince","st","stusps"]].drop_duplicates().reset_index(drop=True)
    g2 = date_tmp['YM'].drop_duplicates().reset_index(drop=True)
    df_plot = g1.merge(g2,how='cross').merge(df_plot,how='left',on=["StateProvince","st","stusps",'YM']).fillna(0)
    df_plot['cumsum'] = df_plot.groupby(["StateProvince","st","stusps"])['TotalIncludingTax'].transform(pd.Series.cumsum)
    
    scl = [0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],\
                        [0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],\
                        [0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]
                        
    fig = px.choropleth(df_plot,locations=df_plot.stusps, 
                        locationmode="USA-states", 
                        color=df_plot['cumsum'], 
                        scope="usa", 
                        labels={"StateProvince":"State","cumsum":"Order Value"},
                        animation_frame="YM",
                        hover_data={
                                    "stusps":False,
                                    'StateProvince':True,
                                    "TotalIncludingTax":False,
                                    "cumsum":":,.2f"},
                        range_color=[0,df_plot['cumsum'].max()],
                        color_continuous_scale=scl
                        )
    fig.update_layout(
        title = 'Total Order Value by State',
        title_x = 0.5,
        title_y = 0.95,
        font={"size":14}, 
        geo = dict(
            showland = True,
            landcolor = "rgb(212, 212, 212)",
            subunitcolor = "rgb(255, 255, 255)",
            countrycolor = "rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "rgb(255, 255, 255)",
            showsubunits = True,
            showcountries = True,
            resolution = 110,
        ))
    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 10
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
    fig2 = go.Figure()
    for tr in fig.frames[-1].data:
        fig2.add_trace(tr)
    fig2.layout = fig.layout
    fig2.frames = fig.frames
    fig2.layout['sliders'][0]['active'] = len(fig.frames) - 1
    return fig2

#### Graph 2

In [346]:
@app.callback(
    Output("graph2_tab2","figure"),
    State('dropdown1_tab1', 'value'), 
    State('dropdown2_tab1', 'value'), 
    State('dropdown3_tab1', 'value'), 
    State('dropdown4_tab1', 'value'), 
    State('dropdown5_tab1', 'value'), 
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    State("dropdown_tab2","value"),
    State("checklist_tab2","value"),
    Input('apply-button','n_clicks'),
)    
def update_graph(v1,v2,v3,v4,v5,sdate,edate,vtab2_1,vtab2_2,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_city_new.copy()
    if len(v1) != 0:
        temp1 = temp1[temp1["Region"].isin(v1)]
    if len(v2) != 0:    
        temp1 = temp1[temp1["Subregion"].isin(v2)]
    if len(v3) != 0:    
        temp1 = temp1[temp1["Country"].isin(v3)]
    if len(v4) != 0:    
        temp1 = temp1[temp1["SalesTerritory"].isin(v4)]
    if len(v5) != 0:    
        temp1 = temp1[temp1["StateProvince"].isin(v5)]
    
    date_tmp = d_date.copy()
    date_tmp = date_tmp[(date_tmp["Date"] >= sdate) & (date_tmp["Date"] <= edate)]
    date_tmp['YM'] = pd.to_datetime(date_tmp["Date"]).dt.strftime("%Y-%m")
    temp2 = f_order.copy()
    temp2 = temp2[(temp2["OrderDateKey"] >= sdate) & (temp2["OrderDateKey"] <= edate)]
    
    temp3 = d_customer.copy()
    if type(vtab2_1) == list: 
        temp3 = temp3[temp3.Category.isin(vtab2_1)]
    else:
        temp3 = temp3[temp3.Category == vtab2_1]
    if len(vtab2_2) == 0:
        raise dash.exceptions.PreventUpdate
    else: 
        temp3 = temp3[temp3.BuyingGroup.isin(vtab2_2)]
    a = temp2.merge(temp1,left_on=temp2.CityHash,right_on=temp1.hash)[["StateProvince","st","stusps","CustomerHash",'OrderDateKey']]
    b = a.merge(temp3,left_on=a.CustomerHash,right_on=temp3.hash)[["Customer","StateProvince","st","stusps",'OrderDateKey']]
    c = b.groupby(["StateProvince","st","stusps",'OrderDateKey']).count().reset_index().rename(columns={"Customer":"Num_of_Cust"})
    c['YM'] = pd.to_datetime(c["OrderDateKey"]).dt.strftime("%Y-%m")
    df_plot = c.groupby(["StateProvince","st","stusps",'YM']).sum().reset_index()
    g1 = df_plot[["StateProvince","st","stusps"]].drop_duplicates().reset_index(drop=True)
    g2 = date_tmp['YM'].drop_duplicates().reset_index(drop=True)
    df_plot = g1.merge(g2,how='cross').merge(df_plot,how='left',on=["StateProvince","st","stusps",'YM']).fillna(0)
    df_plot['cumsum'] = df_plot.groupby(["StateProvince","st","stusps"])['Num_of_Cust'].transform(pd.Series.cumsum)
    
    scl = [0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],\
                        [0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],\
                        [0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]
                        
    fig = px.choropleth(df_plot,locations=df_plot.stusps, 
                        locationmode="USA-states", 
                        color=df_plot['cumsum'], 
                        scope="usa", 
                        labels={"StateProvince":"State","cumsum":"Number of Customers"},
                        animation_frame="YM",
                        hover_data={
                                    "stusps":False,
                                    'StateProvince':True,
                                    "Num_of_Cust":False,
                                    "cumsum":":,.2f"},
                        range_color=[0,df_plot['cumsum'].max()],
                        color_continuous_scale=scl
                        )
    fig.update_layout(
        title = 'Number of Customers by State',
        title_x = 0.5,
        title_y = 0.95,
        font={"size":14}, 
        geo = dict(
            showland = True,
            landcolor = "rgb(212, 212, 212)",
            subunitcolor = "rgb(255, 255, 255)",
            countrycolor = "rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "rgb(255, 255, 255)",
            showsubunits = True,
            showcountries = True,
            resolution = 110,
        ))
    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 10
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
    fig2 = go.Figure()
    for tr in fig.frames[-1].data:
        fig2.add_trace(tr)
    fig2.layout = fig.layout
    fig2.frames = fig.frames
    fig2.layout['sliders'][0]['active'] = len(fig.frames) - 1
    return fig2

#### Graph 3

In [347]:
@app.callback(Output('graph3_tab2', 'figure'), 
              State('dropdown1_tab1', 'value'), 
              State('dropdown2_tab1', 'value'), 
              State('dropdown3_tab1', 'value'), 
              State('dropdown4_tab1', 'value'), 
              State('dropdown5_tab1', 'value'), 
              State('daterange', 'start_date'), 
              State('daterange', 'end_date'),
              State("dropdown_tab2","value"),
              State("checklist_tab2","value"),
              State("dropdown_tab4","value"),
              State("dropdownmain_tab6","value"),
              Input('apply-button','n_clicks'))
def update_graph(v1,v2,v3,v4,v5,sdate,edate,vtab2_1,vtab2_2,vtab4,vtab6, n_clicks):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp = d_city.copy()
    if len(v1) != 0:
        temp = temp[temp["Region"].isin(v1)]
    if len(v2) != 0:    
        temp = temp[temp["Subregion"].isin(v2)]
    if len(v3) != 0:    
        temp = temp[temp["Country"].isin(v3)]
    if len(v4) != 0:    
        temp = temp[temp["SalesTerritory"].isin(v4)]
    if len(v5) != 0:    
        temp = temp[temp["StateProvince"].isin(v5)]
    temp = f_order.merge(temp,left_on=f_order.CityHash,right_on=temp.hash)[["OrderDateKey","TotalIncludingTax","CustomerHash","SalespersonHash","PickerHash","StockItemHash"]]
    temp1 = d_customer.copy()
    if type(vtab2_1) == list: 
        temp1 = temp1[temp1.Category.isin(vtab2_1)]
    else:
        temp1 = temp1[temp1.Category == vtab2_1]
    if len(vtab2_2) == 0:
        raise dash.exceptions.PreventUpdate
    else: 
        temp1 = temp1[temp1.BuyingGroup.isin(vtab2_2)]
    temp1 = temp1.merge(temp,left_on=temp1.hash,right_on=temp.CustomerHash)[["OrderDateKey","TotalIncludingTax","SalespersonHash","PickerHash","StockItemHash"]]
    temp2 = d_employee.copy()
    if type(vtab4) == list:
        temp2 = temp2[temp2.Employee.isin(vtab4)]
    else:
        temp2 = temp2[temp2.Employee == vtab4]
    temp2_saler = temp2[temp2.IsSalesperson == True]
    temp2_picker = temp2[temp2.IsSalesperson == False]
    temp2_join1 = temp2_saler.merge(temp1,left_on=temp2_saler.hash,right_on=temp1.SalespersonHash)[["OrderDateKey","TotalIncludingTax","StockItemHash"]]
    temp2_join2 = temp2_picker.merge(temp1,left_on=temp2_picker.hash,right_on=temp1.PickerHash)[["OrderDateKey","TotalIncludingTax","StockItemHash"]]
    temp2 = pd.concat([temp2_join1,temp2_join2]).reset_index(drop=True)

    temp3 = d_stock_item.copy()
    if (type(vtab6) == list) & (len(vtab6) == 0): 
        pass
    elif type(vtab6) == list:
        temp3 = temp3[temp3.StockItem.isin(vtab6)]
    else:
        temp3 = temp3[temp3.StockItem == vtab6]
    temp3 = temp3.merge(temp2,left_on=temp3.hash,right_on=temp2.StockItemHash)

    date_temp = d_date[(d_date["Date"] >= sdate) & (d_date["Date"] <= edate)]
    dff = date_temp.merge(
            temp3.groupby("OrderDateKey").sum()["TotalIncludingTax"],
            how="left",
            left_on=date_temp.Date,
            right_on=temp3.groupby("OrderDateKey").sum()["TotalIncludingTax"].index
        )[["Date","TotalIncludingTax"]]
    dff['YM'] = pd.to_datetime(dff["Date"]).dt.strftime("%Y-%m")
    dff = dff.groupby("YM").sum().reset_index()
    dff["cumsum"] = dff.TotalIncludingTax.cumsum()
    lst = []
    for i in range(len(dff)):
        lst.append(dff.loc[i,"cumsum"]/(i+1))
    dff["Avg"] = lst
        
    fig = px.line(dff,dff.YM,dff.TotalIncludingTax,title="Total Order Value Over Time",labels={"TotalIncludingTax":"Order Value"})
    fig.add_trace(go.Scatter(x=dff.YM,y=dff.Avg,name="Average",line = dict(width=2, dash='dash')))
    fig.update_layout(
        showlegend=False,
        font={"size":14}, 
        yaxis_title=None, 
        xaxis_title=None,
        title_y = 0.95,
        title_x = 0.5,
        hovermode='x unified'
        )
    return fig

#### Graph 4

In [348]:
@app.callback(
    Output("graph4_tab2","figure"),
    State("checklist_tab2","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks')
)
def update_graph(buyinggroup,sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_customer.copy()
    temp1 = temp1[temp1["BuyingGroup"].isin(buyinggroup)]
    a = f_order.merge(temp1,left_on=f_order.CustomerHash, right_on=temp1.hash)[["OrderDateKey","BuyingGroup","TotalIncludingTax"]]
    b = a.groupby(["BuyingGroup","OrderDateKey"]).count().reset_index()
    b.columns = ["BuyingGroup","OrderDateKey","Times"]
    date_temp = d_date[(d_date["Date"] >= sdate) & (d_date["Date"] <= edate)]
    c = b.BuyingGroup.drop_duplicates().to_frame().merge(date_temp.Date,how='cross')
    df_plot = c.merge(b,how='left',left_on=["BuyingGroup","Date"],right_on=["BuyingGroup","OrderDateKey"]).drop("OrderDateKey",axis=1).fillna(0)
    df_plot['YM'] = pd.to_datetime(df_plot["Date"]).dt.strftime("%Y-%m")
    df_plot = df_plot.groupby(["BuyingGroup","YM"]).sum().reset_index()
    fig = px.line(df_plot,x="YM",y="Times",color="BuyingGroup")
    fig.update_layout(
        legend=dict(
            orientation='h',
            yanchor="top",
            y=1.05,
            xanchor="left",
            x=0,
            ),
        font={"size":14}, yaxis_title=None, xaxis_title=None,
        title = "Number of Orders Over Time",title_y = 0.95,title_x = 0.5,
        legend_title_font_size = 12,
        hovermode='x unified'
    )
    return fig

#### Graph 5

In [349]:
@app.callback(
    Output("graph5_tab2","figure"),
    State("dropdown1_tab6","value"),
    State("dropdown2_tab6","value"),
    State("dropdown3_tab6","value"),
    State("dropdown4_tab6","value"),
    State("dropdown5_tab6","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks')
)
def update_graph(v1,v2,v3,v4,v5,sdate,edate,n_clicks):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    dff = d_stock_item.copy()
    dff.Color = dff.Color.apply(str)
    dff.SellingPackage = dff.SellingPackage.apply(str)
    dff.BuyingPackage = dff.BuyingPackage.apply(str)
    dff.Brand = dff.Brand.apply(str)
    dff.Size = dff.Size.apply(str)
    if v1 is not None:
        dff = dff[dff.Color == v1]
    if v2 is not None:
        dff = dff[dff.SellingPackage == v2]
    if v3 is not None:
        dff = dff[dff.BuyingPackage == v3]
    if v4 is not None:
        dff = dff[dff.Brand == v4]
    if v5 is not None:
        dff = dff[dff.Size == v5]
    dff2 = f_order.copy()
    dff2 = dff2[(dff2["OrderDateKey"] >= sdate) & (dff2["OrderDateKey"] <= edate)]
    df_plot = dff2.merge(dff,left_on=dff2.StockItemHash,right_on=dff.hash)[["StockItem","TotalIncludingTax"]]\
            .groupby("StockItem").sum().reset_index().sort_values("TotalIncludingTax",ascending=False)
    if len(df_plot) == 0:
        raise dash.exceptions.PreventUpdate
    elif len(df_plot) < 10:
        df_plot = df_plot.reset_index(drop=True)
    else:
        df_plot = df_plot.head(10).reset_index(drop=True)
    cutoff_long_label(df_plot,"StockItem",20)
    df_plot = df_plot.sort_values("TotalIncludingTax").reset_index(drop=True).reset_index().rename(columns={'index':"idx"})
    fig = px.bar(df_plot,df_plot["TotalIncludingTax"],
                                df_plot.idx,
                                orientation='h',
                                labels={"TotalIncludingTax":"Order Value"},
                                text=df_plot["TotalIncludingTax"],
                                hover_data={"StockItem":True,
                                             'idx':False,
                                             "TotalIncludingTax":":,.2f"
                                            }
                                 )
    fig.update_layout(yaxis=dict(
                tickvals = df_plot.idx,
                ticktext = df_plot["StockItem_short"]
                ),
                xaxis_title=None,
                yaxis_title=None,
                title = "Top 10 Stock Items <br> with Highest Total Order Value",
                title_y = 0.95,
                title_x = 0.5,
                font={"size":14},
    )
    fig.update_xaxes(range=[0,df_plot.TotalIncludingTax.max()*1.2])
    fig.update_traces(textposition='outside',texttemplate="%{value:,.3s}")
    return fig

#### Graph 6

In [350]:
@app.callback(
    Output("graph6_tab2","figure"),
    State("dropdown1_tab6","value"),
    State("dropdown2_tab6","value"),
    State("dropdown3_tab6","value"),
    State("dropdown4_tab6","value"),
    State("dropdown5_tab6","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks')
)
def update_graph(v1,v2,v3,v4,v5,sdate,edate,n_clicks):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    dff = d_stock_item.copy()
    dff.Color = dff.Color.apply(str)
    dff.SellingPackage = dff.SellingPackage.apply(str)
    dff.BuyingPackage = dff.BuyingPackage.apply(str)
    dff.Brand = dff.Brand.apply(str)
    dff.Size = dff.Size.apply(str)
    if v1 is not None:
        dff = dff[dff.Color == v1]
    if v2 is not None:
        dff = dff[dff.SellingPackage == v2]
    if v3 is not None:
        dff = dff[dff.BuyingPackage == v3]
    if v4 is not None:
        dff = dff[dff.Brand == v4]
    if v5 is not None:
        dff = dff[dff.Size == v5]
    dff2 = f_order.copy()
    dff2 = dff2[(dff2["OrderDateKey"] >= sdate) & (dff2["OrderDateKey"] <= edate)]
    df_plot = dff2.merge(dff,left_on=dff2.StockItemHash,right_on=dff.hash)[["StockItem","TotalIncludingTax"]]\
            .groupby("StockItem").sum().reset_index().sort_values("TotalIncludingTax")
    if len(df_plot) == 0:
        raise dash.exceptions.PreventUpdate
    elif len(df_plot) < 10:
        df_plot = df_plot.reset_index(drop=True)
    else:
        df_plot = df_plot.head(10).reset_index(drop=True)
    cutoff_long_label(df_plot,"StockItem",20)
    df_plot = df_plot.sort_values("TotalIncludingTax",ascending=False).reset_index(drop=True).reset_index().rename(columns={'index':"idx"})
    fig = px.bar(df_plot,df_plot["TotalIncludingTax"],
                                df_plot.idx,
                                orientation='h',
                                labels={"TotalIncludingTax":"Order Value"},
                                text=df_plot["TotalIncludingTax"],
                                hover_data={"StockItem":True,
                                             'idx':False,
                                             "TotalIncludingTax":":,.2f"
                                            }
                                 )
    fig.update_layout(yaxis=dict(
                tickvals = df_plot.idx,
                ticktext = df_plot["StockItem_short"]
                ),
                xaxis_title=None,
                yaxis_title=None,
                title = "Top 10 Stock Items <br> with Lowest Total Order Value",
                title_y = 0.95,
                title_x = 0.5,
                font={"size":14},
    )
    fig.update_xaxes(range=[0,df_plot.TotalIncludingTax.max()*1.2])
    fig.update_traces(textposition='outside',texttemplate="%{value:,.3s}")
    return fig

### Plot tab 3

#### Graph 1

In [351]:
@app.callback(
    Output("graph1_tab3","figure"),
    State("dropdown1_tab6","value"),
    State("dropdown2_tab6","value"),
    State("dropdown3_tab6","value"),
    State("dropdown4_tab6","value"),
    State("dropdown5_tab6","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks')
)
def update_graph(v1,v2,v3,v4,v5,sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_stock_item.copy()
    temp1.Color = temp1.Color.apply(str)
    temp1.SellingPackage = temp1.SellingPackage.apply(str)
    temp1.BuyingPackage = temp1.BuyingPackage.apply(str)
    temp1.Brand = temp1.Brand.apply(str)
    temp1.Size = temp1.Size.apply(str)
    if v1 is not None:
        temp1 = temp1[temp1.Color == v1]
    if v2 is not None:
        temp1 = temp1[temp1.SellingPackage == v2]
    if v3 is not None:
        temp1 = temp1[temp1.BuyingPackage == v3]
    if v4 is not None:
        temp1 = temp1[temp1.Brand == v4]
    if v5 is not None:
        temp1 = temp1[temp1.Size == v5]
    temp2 = f_purchase.copy()
    temp2 = temp2[(temp2["DateKey"] >= sdate) & (temp2["DateKey"] <= edate)]       
    a = temp2.merge(temp1,left_on=temp2.StockItemHash,right_on=temp1.hash)[["OrderedQuantity","StockItem"]]
    df_plot = a.groupby("StockItem").sum().reset_index().sort_values("OrderedQuantity",ascending=False).reset_index(drop=True)
    df_plot["pct"] = df_plot.OrderedQuantity / df_plot.OrderedQuantity.sum()
    df_plot["pct"] = df_plot["pct"].apply(lambda x: f"{x:.2%}")
    if len(df_plot) < 15:
        pass
    else:
        df_plot = df_plot.head(15)
    df_plot['parent'] = "Stock Items"

    scl = [0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],\
                            [0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],\
                            [0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]
    fig = go.Figure()
    fig.add_trace(go.Treemap(
        labels=df_plot.StockItem,
        parents=df_plot.parent,
        values=df_plot.OrderedQuantity,
        textinfo = "label+value+text",
        root_color="lightgrey",
        marker_colorscale = scl,
        text=df_plot.pct,
        ))    
    fig.update_layout(
            font={"size":14}, 
            title = "Top 15 Stock Items with Highest Ordered Quantity",
            title_x = 0.5,
            title_y = 0.95,
    )
    return fig   

#### Graph 2

In [352]:
@app.callback(
    Output("graph2_tab3","figure"),
    State("dropdown_tab7","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks')
)
def update_graph(sup,sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_supplier.copy()
    temp1 = temp1[temp1.Supplier.isin(sup)]
    temp2 = f_purchase.copy()
    temp2 = temp2[(temp2["DateKey"] >= sdate) & (temp2["DateKey"] <= edate)] 
    
    a = temp2.merge(temp1,left_on=temp2.SupplierHash,right_on=temp1.hash)[["Supplier","OrderedOuters","OrderedQuantity"]]
    df_plot = a.groupby("Supplier").sum().reset_index()
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Bar(x=df_plot['Supplier'], y=df_plot['OrderedQuantity'],name="Ordered Quantity"))
    fig.add_trace(go.Line(x=df_plot['Supplier'], y=df_plot['OrderedOuters'],name="Ordered Outers"),secondary_y=True)
    fig.update_xaxes(title_text="Supplier")
    fig.update_yaxes(title_text="Ordered Quantity", secondary_y=False)
    fig.update_yaxes(title_text="Ordered Outers", secondary_y=True)
    fig.update_layout(
        font={"size":14}, 
        title = "Ordered Quantity & Ordered Outers by Supplier",
        title_x = 0.5,
        title_y = 0.95,
        legend=dict(
                yanchor="top",
                y=1.03,
                xanchor="right",
                x=0.99
                ),
    )
    return fig

### Plot tab 4

#### Graph 1

In [353]:
@app.callback(Output('graph1_tab4', 'figure'), 
              State('dropdown1_tab1', 'value'), 
              State('dropdown2_tab1', 'value'), 
              State('dropdown3_tab1', 'value'), 
              State('dropdown4_tab1', 'value'), 
              State('dropdown5_tab1', 'value'), 
              State('daterange', 'start_date'), 
              State('daterange', 'end_date'),
              State("dropdown_tab2","value"),
              State("checklist_tab2","value"),
              State("dropdown_tab4","value"),
              State("dropdownmain_tab6","value"),
              Input('apply-button','n_clicks'))
def update_graph(v1,v2,v3,v4,v5,sdate,edate,vtab2_1,vtab2_2,vtab4,vtab6, n_clicks):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp = d_city.copy()
    if len(v1) != 0:
        temp = temp[temp["Region"].isin(v1)]
    if len(v2) != 0:    
        temp = temp[temp["Subregion"].isin(v2)]
    if len(v3) != 0:    
        temp = temp[temp["Country"].isin(v3)]
    if len(v4) != 0:    
        temp = temp[temp["SalesTerritory"].isin(v4)]
    if len(v5) != 0:    
        temp = temp[temp["StateProvince"].isin(v5)]
    temp = f_sale.merge(temp,left_on=f_sale.CityHash,right_on=temp.hash)[["InvoiceDateKey","TotalIncludingTax","Profit","CustomerHash","SalespersonHash","StockItemHash"]]
    temp1 = d_customer.copy()
    if type(vtab2_1) == list: 
        temp1 = temp1[temp1.Category.isin(vtab2_1)]
    else:
        temp1 = temp1[temp1.Category == vtab2_1]
    if len(vtab2_2) == 0:
        raise dash.exceptions.PreventUpdate
    else: 
        temp1 = temp1[temp1.BuyingGroup.isin(vtab2_2)]
    temp1 = temp1.merge(temp,left_on=temp1.hash,right_on=temp.CustomerHash)[["InvoiceDateKey","TotalIncludingTax","Profit","SalespersonHash","StockItemHash"]]
    temp2 = d_employee.copy()
    if type(vtab4) == list:
        temp2 = temp2[temp2.Employee.isin(vtab4)]
    else:
        temp2 = temp2[temp2.Employee == vtab4]
    temp2 = temp2.merge(temp1,left_on=temp2.hash,right_on=temp1.SalespersonHash)[["InvoiceDateKey","TotalIncludingTax","Profit","StockItemHash"]]
    temp3 = d_stock_item.copy()
    if (type(vtab6) == list) & (len(vtab6) == 0): 
        pass
    elif type(vtab6) == list:
        temp3 = temp3[temp3.StockItem.isin(vtab6)]
    else:
        temp3 = temp3[temp3.StockItem == vtab6]
    temp3 = temp3.merge(temp2,left_on=temp3.hash,right_on=temp2.StockItemHash)[["InvoiceDateKey","TotalIncludingTax","Profit"]]
    date_temp = d_date[(d_date["Date"] >= sdate) & (d_date["Date"] <= edate)]
    dff = date_temp.merge(
            temp3.groupby("InvoiceDateKey").sum(),
            how="left",
            left_on=date_temp.Date,
            right_on=temp3.groupby("InvoiceDateKey").sum().index
        )[["Date","TotalIncludingTax","Profit"]]
    dff['YM'] = pd.to_datetime(dff["Date"]).dt.strftime("%Y-%m")
    dff = dff.groupby("YM").sum().reset_index()
    dff.rename(columns={"TotalIncludingTax":"Sales"},inplace=True)
    lst = []
    for i in range(len(dff)):
        if dff.loc[i,"Profit"] == 0:
            lst.append(0)
        else:
            lst.append(dff.loc[i,"Profit"] / dff.loc[i,"Sales"])
    dff["Profit Margin"] = lst
    dff["Profit Margin_"] = dff["Profit Margin"].apply(lambda x: f"{x:.2%}")
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=dff['YM'], y=dff['Sales'],name="Sales"))
    fig.add_trace(go.Scatter(x=dff['YM'], y=dff['Profit'],name="Profit"))
    fig.add_trace(go.Scatter(x=dff['YM'], y=dff['Profit Margin'],name="Profit Margin",line = dict(width=2, dash='dash'),hovertext=dff['Profit Margin_'],hoverinfo="text+x",),secondary_y=True)
    fig.update_yaxes(range=[0,dff.Sales.max()*1.05])
    fig.update_yaxes(range=[0,1], secondary_y=True)
    
    fig.update_layout(
        font={"size":14}, 
        title = "Total Sales & Profit Over Time",
        title_y = 0.95,
        title_x = 0.5,
        hovermode='x unified',
        legend=dict(
                    title = None,
                    yanchor="top",
                    y=1.15,
                    xanchor="right",
                    x=0.99
                    ),
    )
    return fig

#### Graph 2

In [354]:
@app.callback(Output('graph2_tab4', 'figure'), 
              State('dropdown1_tab1', 'value'), 
              State('dropdown2_tab1', 'value'), 
              State('dropdown3_tab1', 'value'), 
              State('dropdown4_tab1', 'value'), 
              State('dropdown5_tab1', 'value'), 
              State('daterange', 'start_date'), 
              State('daterange', 'end_date'),
              State("dropdown_tab2","value"),
              State("checklist_tab2","value"),
              State("dropdown_tab4","value"),
              State("dropdownmain_tab6","value"),
              Input('apply-button','n_clicks'),
              Input("switch_graph_gtab4","value"),
)
def update_graph(v1,v2,v3,v4,v5,sdate,edate,vtab2_1,vtab2_2,vtab4,vtab6,n,type_):
    syear=pd.to_datetime(sdate).year
    eyear=pd.to_datetime(edate).year

    temp = d_city.copy()
    if len(v1) != 0:
        temp = temp[temp["Region"].isin(v1)]
    if len(v2) != 0:    
        temp = temp[temp["Subregion"].isin(v2)]
    if len(v3) != 0:    
        temp = temp[temp["Country"].isin(v3)]
    if len(v4) != 0:    
        temp = temp[temp["SalesTerritory"].isin(v4)]
    if len(v5) != 0:    
        temp = temp[temp["StateProvince"].isin(v5)]
    f_temp = f_sale.copy()
    f_temp["InvoiceDateKey"] = pd.to_datetime(f_temp["InvoiceDateKey"])
    temp = f_temp.merge(temp,left_on=f_temp.CityHash,right_on=temp.hash)[["InvoiceDateKey","TotalIncludingTax","Profit","CustomerHash","SalespersonHash","StockItemHash"]]
    temp1 = d_customer.copy()
    if type(vtab2_1) == list: 
        temp1 = temp1[temp1.Category.isin(vtab2_1)]
    else:
        temp1 = temp1[temp1.Category == vtab2_1]
    if len(vtab2_2) == 0:
        raise dash.exceptions.PreventUpdate
    else: 
        temp1 = temp1[temp1.BuyingGroup.isin(vtab2_2)]
    temp1 = temp1.merge(temp,left_on=temp1.hash,right_on=temp.CustomerHash)[["InvoiceDateKey","TotalIncludingTax","Profit","SalespersonHash","StockItemHash"]]
    temp2 = d_employee.copy()
    if type(vtab4) == list:
        temp2 = temp2[temp2.Employee.isin(vtab4)]
    else:
        temp2 = temp2[temp2.Employee == vtab4]
    temp2 = temp2.merge(temp1,left_on=temp2.hash,right_on=temp1.SalespersonHash)[["InvoiceDateKey","TotalIncludingTax","Profit","StockItemHash"]]
    temp3 = d_stock_item.copy()
    if (type(vtab6) == list) & (len(vtab6) == 0): 
        pass
    elif type(vtab6) == list:
        temp3 = temp3[temp3.StockItem.isin(vtab6)]
    else:
        temp3 = temp3[temp3.StockItem == vtab6]
    temp3 = temp3.merge(temp2,left_on=temp3.hash,right_on=temp2.StockItemHash)[["InvoiceDateKey","TotalIncludingTax","Profit"]]

    date_temp = d_date.copy()
    date_temp["Date"] = pd.to_datetime(date_temp["Date"])
    date_temp = date_temp[(date_temp["Date"].dt.year >= syear) & (date_temp["Date"].dt.year <= eyear)]
    
    dff = date_temp.merge(
                temp3.groupby("InvoiceDateKey").sum(),
                how="left",
                left_on=date_temp.Date,
                right_on=temp3.groupby("InvoiceDateKey").sum().index
            )[["Date","TotalIncludingTax","Profit"]].fillna(0)
    dff['cumsum_s'] = dff.groupby(dff["Date"].dt.year)['TotalIncludingTax'].transform(pd.Series.cumsum)
    dff['cumsum_p'] = dff.groupby(dff["Date"].dt.year)['Profit'].transform(pd.Series.cumsum)
    if type_ == "Sales":
        fig = go.Figure()
        for year in range(eyear,syear-1,-1):
            if year == eyear:
                fig.add_trace(go.Scatter(
                    x=dff[dff["Date"].dt.year == eyear].Date,
                    y=dff[dff["Date"].dt.year == year]['cumsum_s'],
                    name=f"Cumulative Sales in {year}", 
                    fill='tozeroy',
                    hovertemplate="%{y:.3s}",
                    ))
            else:
                fig.add_trace(go.Scatter(
                    x=dff[dff["Date"].dt.year == eyear].Date,
                    y=dff[dff["Date"].dt.year == year]['cumsum_s'],
                    name=f"Cumulative Sales in {year}", 
                    fill='tonexty',
                    text=year,
                    hovertemplate="%{y:.3s}",
                    ))
        fig.update_layout(
                font={"size":14}, 
                title = "Cumulative Yearly Sales Comparisons",
                title_y = 0.95,
                title_x = 0.5,
                hovermode='x unified',
                legend=dict(
                            title = None,
                            orientation='h',
                            yanchor="top",
                            y=1.15,
                            xanchor="left",
                            x=0
                            ),
            )
    else:
        fig = go.Figure()
        for year in range(eyear,syear-1,-1):
            if year == eyear:
                fig.add_trace(go.Scatter(
                    x=dff[dff["Date"].dt.year == eyear].Date,
                    y=dff[dff["Date"].dt.year == year]['cumsum_p'],
                    name=f"Cumulative Profit in {year}", 
                    fill='tozeroy',
                    hovertemplate="%{y:.3s}",
                    ))
            else:
                fig.add_trace(go.Scatter(
                    x=dff[dff["Date"].dt.year == eyear].Date,
                    y=dff[dff["Date"].dt.year == year]['cumsum_p'],
                    name=f"Cumulative Profit in {year}", 
                    fill='tonexty',
                    text=year,
                    hovertemplate="%{y:.3s}",
                    ))
        fig.update_layout(
                font={"size":14}, 
                title = "Cumulative Yearly Profit Comparisons",
                title_y = 0.95,
                title_x = 0.5,
                hovermode='x unified',
                legend=dict(
                            title = None,
                            orientation='h',
                            yanchor="top",
                            y=1.15,
                            xanchor="left",
                            x=0
                            ),
            )    
    return fig

#### Graph 3

In [355]:
@app.callback(Output('graph3_tab4', 'figure'), 
              State('dropdown1_tab1', 'value'), 
              State('dropdown2_tab1', 'value'), 
              State('dropdown3_tab1', 'value'), 
              State('dropdown4_tab1', 'value'), 
              State('dropdown5_tab1', 'value'), 
              State('daterange', 'start_date'), 
              State('daterange', 'end_date'),
              State("dropdown_tab2","value"),
              State("checklist_tab2","value"),
              State("dropdown_tab4","value"),
              State("dropdownmain_tab6","value"),
              Input('apply-button','n_clicks'))
def update_graph(v1,v2,v3,v4,v5,sdate,edate,vtab2_1,vtab2_2,vtab4,vtab6, n_clicks):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp = d_city.copy()
    if len(v1) != 0:
        temp = temp[temp["Region"].isin(v1)]
    if len(v2) != 0:    
        temp = temp[temp["Subregion"].isin(v2)]
    if len(v3) != 0:    
        temp = temp[temp["Country"].isin(v3)]
    if len(v4) != 0:    
        temp = temp[temp["SalesTerritory"].isin(v4)]
    if len(v5) != 0:    
        temp = temp[temp["StateProvince"].isin(v5)]
    temp = f_sale.merge(temp,left_on=f_sale.CityHash,right_on=temp.hash)[["InvoiceDateKey","TotalIncludingTax","Profit","CustomerHash","SalespersonHash","StockItemHash"]]
    temp1 = d_customer.copy()
    if type(vtab2_1) == list: 
        temp1 = temp1[temp1.Category.isin(vtab2_1)]
    else:
        temp1 = temp1[temp1.Category == vtab2_1]
    if len(vtab2_2) == 0:
        raise dash.exceptions.PreventUpdate
    else: 
        temp1 = temp1[temp1.BuyingGroup.isin(vtab2_2)]
    temp1 = temp1.merge(temp,left_on=temp1.hash,right_on=temp.CustomerHash)[["InvoiceDateKey","TotalIncludingTax","Profit","SalespersonHash","StockItemHash"]]
    temp2 = d_employee.copy()
    if type(vtab4) == list:
        temp2 = temp2[temp2.Employee.isin(vtab4)]
    else:
        temp2 = temp2[temp2.Employee == vtab4]
    temp2 = temp2.merge(temp1,left_on=temp2.hash,right_on=temp1.SalespersonHash)[["InvoiceDateKey","TotalIncludingTax","Profit","StockItemHash"]]
    temp3 = d_stock_item.copy()
    if (type(vtab6) == list) & (len(vtab6) == 0): 
        pass
    elif type(vtab6) == list:
        temp3 = temp3[temp3.StockItem.isin(vtab6)]
    else:
        temp3 = temp3[temp3.StockItem == vtab6]
    temp3 = temp3.merge(temp2,left_on=temp3.hash,right_on=temp2.StockItemHash)[["InvoiceDateKey","TotalIncludingTax","Profit"]]
    date_temp = d_date[(d_date["Date"] >= sdate) & (d_date["Date"] <= edate)]
    dff = date_temp.merge(
            temp3.groupby("InvoiceDateKey").sum(),
            how="left",
            left_on=date_temp.Date,
            right_on=temp3.groupby("InvoiceDateKey").sum().index
        )[["Date","TotalIncludingTax","Profit"]]
    dff['YM'] = pd.to_datetime(dff["Date"]).dt.strftime("%Y-%m")
    dff = dff.groupby("YM").sum().reset_index()
    dff.rename(columns={"TotalIncludingTax":"Sales"},inplace=True)
    lst = []
    for i in range(len(dff)):
        if dff.loc[i,"Profit"] == 0:
            lst.append(0)
        else:
            lst.append(dff.loc[i,"Profit"] / dff.loc[i,"Sales"])
    dff["Profit Margin"] = lst
    dff["Profit Margin_"] = dff["Profit Margin"].apply(lambda x: f"{x:.2%}")
    dff["Lifetime Profit Margin"] = f_sale.Profit.sum()/f_sale.TotalIncludingTax.sum()
    dff["Lifetime Profit Margin_"] = dff["Lifetime Profit Margin"].apply(lambda x: f"{x:.2%}")
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dff['YM'], y=dff['Profit Margin'],name="Profit Margin",hovertext=dff['Profit Margin_'],hoverinfo="text+x",))
    fig.add_trace(go.Scatter(x=dff['YM'], y=dff['Lifetime Profit Margin'],name="Lifetime Profit Margin",hovertext=dff['Lifetime Profit Margin_'],hoverinfo="text+x"))
    
    fig.update_layout(
        font={"size":14}, 
        title = "Profit Margin vs Lifetime Profit Margin",
        title_y = 0.95,
        title_x = 0.5,
        hovermode='x unified',
        legend=dict(
                    title = None,
                    orientation='h',
                    yanchor="top",
                    y=1.15,
                    xanchor="left",
                    x=0
                    ),
    )
    return fig

#### Graph 4

In [356]:
@app.callback(
    Output('graph4_tab4', 'figure'),
    State("dropdown_tab2","value"),
    State("checklist_tab2","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks'),
)
def update_graph(vtab2_1,vtab2_2,sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_customer.copy()
    if type(vtab2_1) == list: 
        temp1 = temp1[temp1.Category.isin(vtab2_1)]
    else:
        temp1 = temp1[temp1.Category == vtab2_1]
    if len(vtab2_2) == 0:
        raise dash.exceptions.PreventUpdate
    else: 
        temp1 = temp1[temp1.BuyingGroup.isin(vtab2_2)]
    temp2 = f_sale.copy()
    temp2 = temp2[(temp2["InvoiceDateKey"] >= sdate) & (temp2["InvoiceDateKey"] <= edate)]

    a = temp2.merge(temp1,left_on=temp2.CustomerHash,right_on=temp1.hash)[["TotalDryItems","TotalChillerItems","BuyingGroup","Category"]]
    a.loc[a.Category != "NoveltyShop","Category"] = "Other"
    b = a.groupby(["BuyingGroup","Category"]).sum().reset_index()
    df_plot = pd.melt(b,id_vars=["BuyingGroup","Category"],value_vars=["TotalDryItems","TotalChillerItems"],var_name='Type',value_name="Quantity")
    tmp = df_plot.groupby(["BuyingGroup","Category"]).sum()["Quantity"].reset_index().rename(columns={"Quantity":"Total"})
    df_plot = df_plot.merge(tmp,on=["BuyingGroup","Category"])
    df_plot['Percentage'] = df_plot.Quantity / df_plot.Total
    df_plot['Percentage'] = df_plot['Percentage'].apply(lambda x: '{:.2%}'.format(x))
    df_plot.Type = df_plot.Type.replace("TotalDryItems","Dry Items")
    df_plot.Type = df_plot.Type.replace("TotalChillerItems","Chiller Items")
    fig = px.bar(df_plot, x="Category", y="Quantity", facet_col="BuyingGroup", color="Type",labels={"Category":""},text="Percentage")
    fig.update_layout(
        title="Total Sales Quantity by Customers",
        font={"size":14},
        title_y = 0.95,
        title_x = 0.5,
        legend=dict(
                    title = None,
                    yanchor="top",
                    y=0.99,
                    xanchor="right",
                    x=0.99
                    ),
    )
    fig.update_traces(textposition='inside')
    return fig

#### Graph 5

In [357]:
@app.callback(
    Output("graph5_tab4","figure"),
    State('dropdown1_tab1', 'value'), 
    State('dropdown2_tab1', 'value'), 
    State('dropdown3_tab1', 'value'), 
    State('dropdown4_tab1', 'value'), 
    State('dropdown5_tab1', 'value'), 
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks'),
    Input("switch_graph_gtab4","value"),
)
def update_graph(v1,v2,v3,v4,v5,sdate,edate,n,graph):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_city_new.copy()
    if len(v1) != 0:
        temp1 = temp1[temp1["Region"].isin(v1)]
    if len(v2) != 0:    
        temp1 = temp1[temp1["Subregion"].isin(v2)]
    if len(v3) != 0:    
        temp1 = temp1[temp1["Country"].isin(v3)]
    if len(v4) != 0:    
        temp1 = temp1[temp1["SalesTerritory"].isin(v4)]
    if len(v5) != 0:    
        temp1 = temp1[temp1["StateProvince"].isin(v5)]

    date_tmp = d_date.copy()
    date_tmp = date_tmp[(date_tmp["Date"] >= sdate) & (date_tmp["Date"] <= edate)]
    date_tmp['YM'] = pd.to_datetime(date_tmp["Date"]).dt.strftime("%Y-%m")
    temp2 = f_sale.copy()
    temp2 = temp2[(temp2["InvoiceDateKey"] >= sdate) & (temp2["InvoiceDateKey"] <= edate)] 

    a = temp2[["CityHash","TotalIncludingTax","Profit","InvoiceDateKey"]].merge(temp1,left_on=temp2[["CityHash","TotalIncludingTax","Profit"]].CityHash,right_on=temp1.hash)[["StateProvince","st","stusps","TotalIncludingTax","Profit",'InvoiceDateKey']]
    c1 = a.groupby(["StateProvince","st","stusps",'InvoiceDateKey']).sum().reset_index().sort_values("TotalIncludingTax").reset_index(drop=True)
    c1['YM'] = pd.to_datetime(c1["InvoiceDateKey"]).dt.strftime("%Y-%m")
    df_plot1 = c1.groupby(["StateProvince","st","stusps",'YM']).sum().reset_index()
    g1 = df_plot1[["StateProvince","st","stusps"]].drop_duplicates().reset_index(drop=True)
    g2 = date_tmp['YM'].drop_duplicates().reset_index(drop=True)
    df_plot1 = g1.merge(g2,how='cross').merge(df_plot1,how='left',on=["StateProvince","st","stusps",'YM']).fillna(0)
    df_plot1['cumsum'] = df_plot1.groupby(["StateProvince","st","stusps"])['TotalIncludingTax'].transform(pd.Series.cumsum)

    c2 = a.groupby(["StateProvince","st","stusps",'InvoiceDateKey']).sum().reset_index().sort_values("Profit").reset_index(drop=True)
    c2['YM'] = pd.to_datetime(c2["InvoiceDateKey"]).dt.strftime("%Y-%m")
    df_plot2 = c2.groupby(["StateProvince","st","stusps",'YM']).sum().reset_index()
    g1_ = df_plot2[["StateProvince","st","stusps"]].drop_duplicates().reset_index(drop=True)
    g2_ = date_tmp['YM'].drop_duplicates().reset_index(drop=True)
    df_plot2 = g1_.merge(g2_,how='cross').merge(df_plot2,how='left',on=["StateProvince","st","stusps",'YM']).fillna(0)
    df_plot2['cumsum'] = df_plot2.groupby(["StateProvince","st","stusps"])['Profit'].transform(pd.Series.cumsum)

    scl = [0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],\
                        [0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],\
                        [0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

    if graph == "Sales":                    
        fig = px.choropleth(df_plot1,locations=df_plot1.stusps, 
                            locationmode="USA-states", 
                            color=df_plot1['cumsum'], 
                            scope="usa", 
                            labels={"StateProvince":"State","cumsum":"Sales"},
                            animation_frame="YM",
                            hover_data={
                                        "stusps":False,
                                        'StateProvince':True,
                                        "cumsum":":,.2f"},
                            range_color=[0,df_plot1['cumsum'].max()],
                            color_continuous_scale=scl
                            )
        fig.update_layout(
            title = 'Total Sales by State',
            title_x = 0.5,
            title_y = 0.95,
            font={"size":14}, 
            geo = dict(
                showland = True,
                landcolor = "rgb(212, 212, 212)",
                subunitcolor = "rgb(255, 255, 255)",
                countrycolor = "rgb(255, 255, 255)",
                showlakes = True,
                lakecolor = "rgb(255, 255, 255)",
                showsubunits = True,
                showcountries = True,
                resolution = 110,
            ))
        fig2 = go.Figure()
        for tr in fig.frames[-1].data:
            fig2.add_trace(tr)
        fig2.layout = fig.layout
        fig2.frames = fig.frames
        fig2.layout['sliders'][0]['active'] = len(fig.frames) - 1
        fig2.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 20
        fig2.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
        return fig2
    else:
        fig = px.choropleth(df_plot2,locations=df_plot2.stusps, 
                            locationmode="USA-states", 
                            color=df_plot2['cumsum'], 
                            scope="usa", 
                            labels={"StateProvince":"State","cumsum":"Profit"},
                            animation_frame="YM",
                            hover_data={
                                        "stusps":False,
                                        'StateProvince':True,
                                        "cumsum":":,.2f"},
                            range_color=[0,df_plot2['cumsum'].max()],
                            color_continuous_scale=scl
                            )
        fig.update_layout(
            title = 'Total Profit by State',
            title_x = 0.5,
            title_y = 0.95,
            font={"size":14}, 
            geo = dict(
                showland = True,
                landcolor = "rgb(212, 212, 212)",
                subunitcolor = "rgb(255, 255, 255)",
                countrycolor = "rgb(255, 255, 255)",
                showlakes = True,
                lakecolor = "rgb(255, 255, 255)",
                showsubunits = True,
                showcountries = True,
                resolution = 110,
            ))
        fig2 = go.Figure()
        for tr in fig.frames[-1].data:
            fig2.add_trace(tr)
        fig2.layout = fig.layout
        fig2.frames = fig.frames
        fig2.layout['sliders'][0]['active'] = len(fig.frames) - 1
        fig2.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 20
        fig2.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
        return fig2

### Plot tab 5

#### Graph 1

In [358]:
@app.callback(
    Output("graph1_tab5","figure"),
    State("dropdown1_tab6","value"),
    State("dropdown2_tab6","value"),
    State("dropdown3_tab6","value"),
    State("dropdown4_tab6","value"),
    State("dropdown5_tab6","value"),
    Input('apply-button','n_clicks')
)
def update_graph(v1,v2,v3,v4,v5,n):
    temp1 = d_stock_item.copy()
    temp1.Color = temp1.Color.apply(str)
    temp1.SellingPackage = temp1.SellingPackage.apply(str)
    temp1.BuyingPackage = temp1.BuyingPackage.apply(str)
    temp1.Brand = temp1.Brand.apply(str)
    temp1.Size = temp1.Size.apply(str)
    if v1 is not None:
        temp1 = temp1[temp1.Color == v1]
    if v2 is not None:
        temp1 = temp1[temp1.SellingPackage == v2]
    if v3 is not None:
        temp1 = temp1[temp1.BuyingPackage == v3]
    if v4 is not None:
        temp1 = temp1[temp1.Brand == v4]
    if v5 is not None:
        temp1 = temp1[temp1.Size == v5]
    a = f_stockholding.merge(temp1,left_on=f_stockholding.StockItemHash,right_on=temp1.hash)[["QuantityOnHand","LastStocktakeQuantity","Brand","IsChillerStock"]]
    lst = []
    for i in range(len(a)):
        if a.loc[i,"IsChillerStock"] == True:
            lst.append("Chiller Stocks")
        else:
            lst.append("Non-Chiller Stocks")
    a["Type"] = lst
    a.drop("IsChillerStock",axis=1,inplace=True)
    df_plot = pd.melt(a, id_vars=['Type',"Brand"],value_vars=["QuantityOnHand",'LastStocktakeQuantity'])
    df_plot = df_plot.sort_values(['Type',"Brand"])
    fig = px.sunburst(df_plot, path=["Brand",'Type',"variable"], values='value',color='Type')
    fig.update_layout(
        font={"size":14}, 
        title = "Quantity On Hand & Last Stocktake Quantity of<br>Chiller and Non-Chiller Stocks",
        title_x = 0.5,
        title_y = 0.95,
    )
    return fig

#### Graph 2

In [359]:
@app.callback(
    Output("graph2_tab5","figure"),
    State("dropdown1_tab6","value"),
    State("dropdown2_tab6","value"),
    State("dropdown3_tab6","value"),
    State("dropdown4_tab6","value"),
    State("dropdown5_tab6","value"),
    Input('apply-button','n_clicks')
)
def update_graph(v1,v2,v3,v4,v5,n):
    temp1 = d_stock_item.copy()
    temp1.Color = temp1.Color.apply(str)
    temp1.SellingPackage = temp1.SellingPackage.apply(str)
    temp1.BuyingPackage = temp1.BuyingPackage.apply(str)
    temp1.Brand = temp1.Brand.apply(str)
    temp1.Size = temp1.Size.apply(str)
    if v1 is not None:
        temp1 = temp1[temp1.Color == v1]
    if v2 is not None:
        temp1 = temp1[temp1.SellingPackage == v2]
    if v3 is not None:
        temp1 = temp1[temp1.BuyingPackage == v3]
    if v4 is not None:
        temp1 = temp1[temp1.Brand == v4]
    if v5 is not None:
        temp1 = temp1[temp1.Size == v5]
    a = f_stockholding.merge(temp1,left_on=f_stockholding.StockItemHash,right_on=temp1.hash)[["StockItem","QuantityOnHand","LastStocktakeQuantity"]]
    df_plot = a.groupby("StockItem").sum().reset_index().sort_values("QuantityOnHand",ascending=False).reset_index(drop=True)
    if len(df_plot) > 10:
        df_plot = df_plot.head(10)
    cutoff_long_label(df_plot,"StockItem",15)
    df_plot = df_plot.reset_index().rename(columns={'index':"idx"})

        
    a1 = f_stockholding.merge(temp1,left_on=f_stockholding.StockItemHash,right_on=temp1.hash)[["StockItem","QuantityOnHand","LastStocktakeQuantity"]]
    df_plot1 = a1.groupby("StockItem").sum().reset_index().sort_values("LastStocktakeQuantity",ascending=False).reset_index(drop=True)
    if len(df_plot1) > 10:
        df_plot1 = df_plot1.head(10)
    cutoff_long_label(df_plot1,"StockItem",15)
    df_plot1 = df_plot1.reset_index().rename(columns={'index':"idx"})

    
    fig = make_subplots(rows=1,cols=2,
                        subplot_titles=("with Highest Quantity On Hand", "with Highest Last Stocktake Quantity"),
                        specs=[[{"secondary_y": True},{"secondary_y": True}]],
                        )
    fig.add_trace(go.Bar(x=df_plot['idx'], 
                        y=df_plot['QuantityOnHand'],
                        name="Quantity On Hand",
                        hovertext=df_plot['StockItem'],
                        hoverinfo="text+y",
                        ),
                row=1, col=1)
    fig.add_trace(go.Line(x=df_plot['idx'], 
                        y=df_plot['LastStocktakeQuantity'],
                        name="Last Stocktake Quantity",
                        hovertext=df_plot['StockItem'],
                        hoverinfo="text+y",
                        ),
                row=1, col=1,secondary_y=True)

    fig.add_trace(go.Bar(x=df_plot1['idx'], 
                        y=df_plot1['QuantityOnHand'],
                        name="Quantity On Hand",
                        hovertext=df_plot1['StockItem'],
                        hoverinfo="text+y",
                        ),
                row=1, col=2)
    fig.add_trace(go.Line(x=df_plot1['idx'], 
                        y=df_plot1['LastStocktakeQuantity'],
                        name="Last Stocktake Quantity",
                        hovertext=df_plot1['StockItem'],
                        hoverinfo="text+y",
                        ),
                row=1, col=2,secondary_y=True)

    fig.update_yaxes(title_text="Quantity On Hand", secondary_y=False,row=1,col=1)
    fig.update_yaxes(title_text="Last Stocktake Quantity", secondary_y=True,row=1,col=2)

    fig.update_xaxes(tickvals = df_plot.idx,ticktext = df_plot["StockItem_short"],row=1,col=1)
    fig.update_xaxes(tickvals = df_plot1.idx,ticktext = df_plot1["StockItem_short"],row=1,col=2)

    fig.update_layout(
        font={"size":14}, 
        title = "Quantity On Hand & Last Stocktake Quantity of Top 10 Stock Items",
        title_x = 0.5,
        title_y = 0.95,
        showlegend=False
    )
    return fig

#### Graph 3

In [360]:
@app.callback(
    Output("graph3_tab5","figure"),
    State("dropdown1_tab6","value"),
    State("dropdown2_tab6","value"),
    State("dropdown3_tab6","value"),
    State("dropdown4_tab6","value"),
    State("dropdown5_tab6","value"),
    Input('apply-button','n_clicks')
)
def update_graph(v1,v2,v3,v4,v5,n):
    temp1 = d_stock_item.copy()
    temp1.Color = temp1.Color.apply(str)
    temp1.SellingPackage = temp1.SellingPackage.apply(str)
    temp1.BuyingPackage = temp1.BuyingPackage.apply(str)
    temp1.Brand = temp1.Brand.apply(str)
    temp1.Size = temp1.Size.apply(str)
    if v1 is not None:
        temp1 = temp1[temp1.Color == v1]
    if v2 is not None:
        temp1 = temp1[temp1.SellingPackage == v2]
    if v3 is not None:
        temp1 = temp1[temp1.BuyingPackage == v3]
    if v4 is not None:
        temp1 = temp1[temp1.Brand == v4]
    if v5 is not None:
        temp1 = temp1[temp1.Size == v5]
    a = f_stockholding.merge(temp1,left_on=f_stockholding.StockItemHash,right_on=temp1.hash)[["StockItem","QuantityOnHand","LastStocktakeQuantity"]]
    df_plot = a.groupby("StockItem").sum().reset_index().sort_values("QuantityOnHand").reset_index(drop=True)
    if len(df_plot) > 10:
        df_plot = df_plot.head(10)
    cutoff_long_label(df_plot,"StockItem",15)
    df_plot = df_plot.reset_index().rename(columns={'index':"idx"})

        
    a1 = f_stockholding.merge(temp1,left_on=f_stockholding.StockItemHash,right_on=temp1.hash)[["StockItem","QuantityOnHand","LastStocktakeQuantity"]]
    df_plot1 = a1.groupby("StockItem").sum().reset_index().sort_values("LastStocktakeQuantity").reset_index(drop=True)
    if len(df_plot1) > 10:
        df_plot1 = df_plot1.head(10)
    cutoff_long_label(df_plot1,"StockItem",15)
    df_plot1 = df_plot1.reset_index().rename(columns={'index':"idx"})

    
    fig = make_subplots(rows=1,cols=2,
                        subplot_titles=("with Lowest Quantity On Hand", "with Lowest Last Stocktake Quantity"),
                        specs=[[{"secondary_y": True},{"secondary_y": True}]],
                        )
    fig.add_trace(go.Bar(x=df_plot['idx'], 
                        y=df_plot['QuantityOnHand'],
                        name="Quantity On Hand",
                        hovertext=df_plot['StockItem'],
                        hoverinfo="text+y",
                        ),
                row=1, col=1)
    fig.add_trace(go.Line(x=df_plot['idx'], 
                        y=df_plot['LastStocktakeQuantity'],
                        name="Last Stocktake Quantity",
                        hovertext=df_plot['StockItem'],
                        hoverinfo="text+y",
                        ),
                row=1, col=1,secondary_y=True)

    fig.add_trace(go.Bar(x=df_plot1['idx'], 
                        y=df_plot1['QuantityOnHand'],
                        name="Quantity On Hand",
                        hovertext=df_plot1['StockItem'],
                        hoverinfo="text+y",
                        ),
                row=1, col=2)
    fig.add_trace(go.Line(x=df_plot1['idx'], 
                        y=df_plot1['LastStocktakeQuantity'],
                        name="Last Stocktake Quantity",
                        hovertext=df_plot1['StockItem'],
                        hoverinfo="text+y",
                        ),
                row=1, col=2,secondary_y=True)

    fig.update_yaxes(title_text="Quantity On Hand", secondary_y=False,row=1,col=1)
    fig.update_yaxes(title_text="Last Stocktake Quantity", secondary_y=True,row=1,col=2)

    fig.update_xaxes(tickvals = df_plot.idx,ticktext = df_plot["StockItem_short"],row=1,col=1)
    fig.update_xaxes(tickvals = df_plot1.idx,ticktext = df_plot1["StockItem_short"],row=1,col=2)

    fig.update_layout(
        font={"size":14}, 
        title = "Quantity On Hand & Last Stocktake Quantity of Top 10 Stock Items",
        title_x = 0.5,
        title_y = 0.95,
        showlegend=False
    )
    return fig

### Plot tab 6

#### Graph 1

In [361]:
@app.callback(
    Output("graph1_tab6","figure"),
    State("dropdown_tab8","value"),
    State("dropdown_tab5","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks')
)
def update_graph(trans_type,pm_type,sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_transaction_type.copy()
    if type(trans_type) == list:
        temp1 = temp1[temp1.TransactionType.isin(trans_type)]
    else:
        temp1 = temp1[temp1.TransactionType == trans_type]

    temp2 = f_transaction.copy()
    temp2 = temp2[(temp2["DateKey"] >= sdate) & (temp2["DateKey"] <= edate)] 
    a = temp2.merge(temp1,left_on=temp2.TransactionTypeHash,right_on=temp1.hash)[["TransactionType","TotalIncludingTax","PaymentMethodHash"]]
    temp3 = d_payment_method.copy()
    if type(pm_type) == list:
        temp3 = temp3[temp3.PaymentMethod.isin(pm_type)]
    else:
        temp3 = temp3[temp3.PaymentMethod == pm_type]
    b = a.merge(temp3,left_on=a.PaymentMethodHash,right_on=temp3.hash)[["TransactionType","TotalIncludingTax",'PaymentMethod']]
    df_plot = b.groupby(["TransactionType","PaymentMethod"]).sum().reset_index().sort_values(["PaymentMethod","TransactionType"],ascending=[True,False]).reset_index(drop=True)

    fig = px.line_polar(df_plot, 
                        r="TotalIncludingTax", 
                        theta="TransactionType", 
                        color="PaymentMethod",
                        line_close=True,
                        labels={"TotalIncludingTax":"Transaction Amount"},
                        range_r =[df_plot.TotalIncludingTax.min()*0.95, df_plot.TotalIncludingTax.max()*1.05],
                )
    fig.update_layout(
        legend=dict(
                yanchor="top",
                y=1.15,
                xanchor="right",
                x=1,
                title=None
                ),
        font={"size":14}, 
        title = "Total Transaction Amount of each Transaction Type",
        title_x = 0.5,
        title_y = 0.95,
    )
    return fig

#### Graph 2

In [362]:
@app.callback(
    Output("graph2_tab6","figure"),
    State("dropdown_tab5","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks')
)
def update_graph(pm,sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_payment_method.copy()
    if type(pm) == list:
        temp1 = temp1[temp1.PaymentMethod.isin(pm)]
    else:
        temp1 = temp1[temp1.PaymentMethod == pm]

    temp2 = f_transaction.copy()
    temp2 = temp2[(temp2["DateKey"] >= sdate) & (temp2["DateKey"] <= edate)] 

    a = temp2.merge(d_customer,left_on=temp2.CustomerHash,right_on=d_customer.hash)[["PaymentMethodHash","BuyingGroup","TotalIncludingTax"]]
    b = a.merge(temp1,left_on=a.PaymentMethodHash,right_on=temp1.hash)[["BuyingGroup","TotalIncludingTax","PaymentMethod"]]
    result = waterfall_table(b,"BuyingGroup","PaymentMethod","TotalIncludingTax")
    
    fig = go.Figure()
    fig.add_trace(go.Waterfall(
                x = [result.x1,result.x2], y = result["Changing"],
                measure = result.Measure,
                base = 0,
                text = result["ChangingText"],
                textposition = 'outside',
                ))
    fig.update_layout(
        yaxis=dict(
        range=[result.loc[:(len(result)-2),'Cumsum'].min()-0.1*result.loc[:(len(result)-2),'Cumsum'].min(),
               result.loc[:(len(result)-2),'Cumsum'].max()+0.1*result.loc[:(len(result)-2),'Cumsum'].max()]
    ),
        font={"size":14}, 
        title = "Total Transaction Amount by Customers' Buying Group",
        title_x = 0.5,
        title_y = 0.95,
    )
    return fig

#### Graph 3

In [363]:
@app.callback(Output('graph3_tab6', 'figure'), 
              State('daterange', 'start_date'), 
              State('daterange', 'end_date'),
              State("dropdown_tab2","value"),
              State("checklist_tab2","value"),
              State("dropdown_tab5","value"),
              State("dropdown_tab7","value"),
              State("dropdown_tab8","value"),
              Input('apply-button','n_clicks'))
def update_graph(sdate,edate,vtab2_1,vtab2_2,vtab5,vtab7,vtab8,n_clicks):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_customer.copy()
    if type(vtab2_1) == list: 
        temp1 = temp1[temp1.Category.isin(vtab2_1)]
    else:
        temp1 = temp1[temp1.Category == vtab2_1]
    if len(vtab2_2) == 0:
        raise dash.exceptions.PreventUpdate
    else: 
        temp1 = temp1[temp1.BuyingGroup.isin(vtab2_2)]
    temp1 = temp1.merge(f_transaction,left_on=temp1.hash,right_on=f_transaction.CustomerHash)[["DateKey","TotalIncludingTax","PaymentMethodHash","SupplierHash","TransactionTypeHash"]]
    temp2 = d_payment_method.copy()
    if type(vtab5) == list:
        temp2 = temp2[temp2.PaymentMethod.isin(vtab5)]
    else:
        temp2 = temp2[temp2.PaymentMethod == vtab5]
    temp2 = temp2.merge(temp1,left_on=temp2.hash,right_on=temp1.PaymentMethodHash)[["DateKey","TotalIncludingTax","SupplierHash","TransactionTypeHash"]]
    temp3 = d_supplier.copy()
    if type(vtab7) == list:
        temp3 = temp3[temp3.Supplier.isin(vtab7)]
    else:
        temp3 = temp3[temp3.Supplier == vtab7]
    temp3 = temp3.merge(temp2,left_on=temp3.hash,right_on=temp2.SupplierHash)[["DateKey","TotalIncludingTax","TransactionTypeHash"]]

    temp4 = d_transaction_type.copy()
    if type(vtab8) == list:
        temp4 = temp4[temp4.TransactionType.isin(vtab8)]
    else:
        temp4 = temp4[temp4.TransactionType == vtab8]
    temp4 = temp4.merge(temp3,left_on=temp4.hash,right_on=temp3.TransactionTypeHash)[["DateKey","TotalIncludingTax"]]
    date_temp = d_date[(d_date["Date"] >= sdate) & (d_date["Date"] <= edate)]
    dff = date_temp.merge(
            temp4.groupby("DateKey").sum(),
            how="left",
            left_on=date_temp.Date,
            right_on=temp4.groupby("DateKey").sum().index
        )[["Date","TotalIncludingTax"]]
    dff['YM'] = pd.to_datetime(dff["Date"]).dt.strftime("%Y-%m")
    dff = dff.groupby("YM").sum().reset_index()
    dff.rename(columns={"TotalIncludingTax":"Transaction Amount"},inplace=True)
    dff['cumsum'] = dff["Transaction Amount"].cumsum()

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=dff['YM'], y=dff['Transaction Amount'],name="Transaction Amount"))
    fig.add_trace(go.Scatter(x=dff['YM'], y=dff['cumsum'],name="Total Transaction Amount",line = dict(width=2, dash='dash')),secondary_y=True)
    fig.update_layout(
        font={"size":14}, 
        hovermode='x unified',
        title="Total Transaction Amount Over Time",
        title_y = 0.95,
        title_x = 0.5,
        legend=dict(
                    title = None,
                    yanchor="top",
                    y=1.15,
                    xanchor="right",
                    x=0.99
                    ),
    )
    return fig

#### Graph 4

In [364]:
@app.callback(
    Output("graph4_tab6","figure"),
    State("dropdown_tab5","value"),
    State('daterange', 'start_date'), 
    State('daterange', 'end_date'),
    Input('apply-button','n_clicks')
)
def update_graph(pm,sdate,edate,n):
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)
    temp1 = d_payment_method.copy()
    if type(pm) == list:
        temp1 = temp1[temp1.PaymentMethod.isin(pm)]
    else:
        temp1 = temp1[temp1.PaymentMethod == pm]
        
    temp2 = f_transaction.copy()
    temp2 = temp2[(temp2["DateKey"] >= sdate) & (temp2["DateKey"] <= edate)] 
        
    a = temp2.merge(d_supplier,left_on=temp2.SupplierHash,right_on=d_supplier.hash)[["PaymentMethodHash","Category","TotalIncludingTax"]]
    b = a.merge(temp1,left_on=a.PaymentMethodHash,right_on=temp1.hash)[["Category","TotalIncludingTax","PaymentMethod"]]
    
    result = waterfall_table(b,"Category","PaymentMethod","TotalIncludingTax")
    
    fig = go.Figure()
    fig.add_trace(go.Waterfall(
                x = [result.x1,result.x2], y = result["Changing"],
                measure = result.Measure,
                base = 0,
                text = result["ChangingText"],
                textposition = 'outside',
                ))
    fig.update_layout(
        yaxis=dict(
        range=[result.loc[:(len(result)-2),'Cumsum'].min()-0.1*result.loc[:(len(result)-2),'Cumsum'].min(),
               result.loc[:(len(result)-2),'Cumsum'].max()+0.1*result.loc[:(len(result)-2),'Cumsum'].max()]
    ),
        font={"size":14}, 
        title = "Total Transaction Amount by Suppliers' Category",
        title_x = 0.5,
        title_y = 0.95,
    )
    return fig

# Run

In [365]:
app.run_server(mode="external")

Dash app running on http://127.0.0.1:8050/
